In [1]:
# coding=utf-8
# -*- coding: utf-8 -*-
import os
from os.path import isdir
import linecache
# python自带的模块，缓存读取文件某一行的内容
import numpy as np
import pandas as pd
import math
import copy as copy1
import scipy
import urllib
# from scipy.signal import savgol_filter
import urllib.parse
from urllib.parse import urlparse
import sys
from numpy import loadtxt
from numpy import savetxt
from numpy.linalg import svd

np.set_printoptions(suppress=True)
# 可以使数组打印更漂亮
np.set_printoptions(threshold=np.inf)
# 解决ndarray打印省略问题
# np.set_printoptions(linewidth=20)
# #打印时候每行显示20个

import pylab
from pylab import *
import matplotlib as mpl

mpl.rcParams['font.sans-serif'] = ['Times New Roman']
# ！
mpl.rcParams['axes.unicode_minus'] = False
# 用来正常显示负号
from matplotlib.ticker import FuncFormatter


def to_percent(temp, position):
    return '%1.0f' % (100 * temp) + '%'


# 刻度以%显示，小数点后1位

import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import signal
from scipy import optimize
from scipy import sparse
from scipy.signal import find_peaks_cwt
from scipy.signal import detrend
from scipy.sparse import linalg
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve
from scipy.signal import savgol_filter
from scipy.cluster import hierarchy
from scipy import cluster
# 用于进行层次聚类，话层次聚类图的工具包
from scipy.spatial.distance import cdist
# 计算距离结算公式

from collections import Counter
# 高性能容量数据类型,count计数器
from decimal import *
import sklearn
from sklearn import decomposition as skldec
# 用于主成分分析降维的包
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
# tsne
from sklearn import metrics
from sklearn.metrics import silhouette_score
# 计算轮廓系数
from sklearn.model_selection import GridSearchCV
# 网格搜索&交叉验证
#from sklearn.externals import joblib
import joblib
# 保存、加载分类器模型
from xgboost import XGBClassifier
# xgboost
from skimage import filters
# 基于Otsu的阈值分割方法,无参数去背景


# ROCq曲线AUC值
# from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
# from sklearn.multiclass import OneVsRestClassifier
# from scipy import interp
# 画图-混淆矩阵
import seaborn as sns
from sklearn.metrics import confusion_matrix
# -----------------------
import struct
# Kmeans模型
import sklearn.cluster
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
# 聚类算法距离计算公式
from scipy.spatial.distance import cdist

# 写入word
import docx
import docx.enum
import docx.shared
import docx.enum.text
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import docx.oxml
import docx.oxml.ns
from docx.oxml.ns import qn

import warnings

warnings.filterwarnings("ignore")


# 忽略警告


In [2]:
# airPLS去背景
def WhittakerSmooth(x, w, lambda_):
    X = np.matrix(x)
    m = X.size
    i = np.arange(0, m)
    E = eye(m, format='csc')
    D = E[1:] - E[:-1]
    # numpy.diff() does not work with sparse matrix. This is a workaround.
    W = diags(w, 0, shape=(m, m))
    A = csc_matrix(W + (lambda_ * D.T * D))
    B = csc_matrix(W * X.T)
    background = spsolve(A, B)
    return np.array(background)


# airPLS去背景主程序
def airPLS(x, lambda_, itermax):
    x = np.array(x)
    m = x.shape[0]  # 计算长宽维度
    w = np.ones(m)
    for i in range(1, itermax + 1):
        z = WhittakerSmooth(x, w, lambda_)
        d = x - z
        dssn = np.abs(d[d < 0].sum())
        # ***********************************************除零问题待解决
        # if fabs(dssn) < 1e-15:
        #     dssn = dssn+0.0001
        # print("dssn:",dssn)
        if (dssn < 0.001 * (abs(x)).sum() or i == itermax):
            if (i == itermax):
                print('WARING max iteration reached!')
            break
        w[d >= 0] = 0
        # d>0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        w[d < 0] = np.exp(i * np.abs(d[d < 0]) / dssn)
        w[0] = np.exp(i * (d[d < 0]).max() / dssn)
        w[-1] = w[0]
    z = d
    return z.tolist()


# 无参数调用去背景算法

# 方法一，出现部分bug,如最小长度必须大于101等，选用方法二
# def background_substruction(data):
#     data = np.array(data)
#     polyorder = 3
#     block_size = 101
#     baseline = np.zeros((len(data)))
#     # 定义基线
#     for x in range(int(len(data))):
#         min_half = int(block_size / 2)
#         max_half = int(block_size / 2)
#         if x - min_half < 0:
#             min_half = 0
#         if x + max_half > len(data):
#             max_half = len(data)
#         selected_data = data[x - min_half:x + max_half]
#         # print("长度：",type(selected_data))
#         thre = filters.threshold_otsu(np.asarray((selected_data)))
#         # print("阈值：",thre)
#         # test = selected_data
#         test = selected_data[selected_data < thre]
#         # 剔除小于“thre”阈值的数据点
#         # print("test1:",len(test))
#         test = test[test < np.mean(test)]
#         test = test[test < np.median(test)]
#         # print("test2:", len(test))
#         # print("x",min(test))
#         # baseline[x] = min(test)
#         baseline[x] = np.mean(test)
#         # print("长度：",len(baseline))
#         # print("基线：",baseline)
#     smooth_baseline = signal.savgol_filter(baseline, block_size, polyorder)
#     corrected_data = data - smooth_baseline
#     return corrected_data.tolist()

# 方法二，ALS交替最小二乘算法
def molification_smoothing(rawspectrum, struct_el, number_of_molifications):
    molifier_kernel = np.linspace(-1, 1, num=struct_el)
    # print(molifier_kernel)
    # 创建-1到1之间数字序列，步长为struct_el
    molifier_kernel[1:-1] = np.exp(-1 / (1 - molifier_kernel[1:-1] ** 2))
    # print(molifier_kernel)
    molifier_kernel[0] = 0
    molifier_kernel[-1] = 0
    molifier_kernel = molifier_kernel / np.sum(molifier_kernel)
    # 列表归一化
    denominormtor = np.convolve(np.ones_like(rawspectrum), molifier_kernel, 'same')
    # np.convolve卷积运算，same：返回的数组长度为max(M, N),边际效应依旧存在
    # np.ones_like，全1数组
    smoothline = rawspectrum
    i = 0
    for i in range(number_of_molifications):
        smoothline = np.convolve(smoothline, molifier_kernel, 'same') / denominormtor
        i += 1
    return smoothline


# 方法二，ALS交替最小二乘算法主函数
def background_substruction(y):
    als_lambda = 3e4
    als_p_weight = 5e-4
    _iterations = 16
    zero_step_struct_el = np.int(2 * np.round(len(y) / 200) + 3)
    """
    猜测，设置步长每10个点为一步，步长越短，拟合效果越保型，最终点数至少大于等于3，所以这里将“+1”该为“+3”
    """
    # print(zero_step_struct_el)
    y_sm = molification_smoothing(y, zero_step_struct_el, 16)
    # 交替（或迭代）做平滑,平滑效果其实不好，主要为了拟合光谱形状
    # plt.plot(y,"r")
    # plt.plot(y_sm,"b")
    # plt.show()
    # compute the derivatives:
    y_sm_1d = np.gradient(y_sm)
    # !!!!!!重要理解 https://docs.scipy.org/doc/numpy-1.12.0/reference/generated/numpy.gradient.html
    # numpy.gradient，返回 N 维数组的梯度，
    # 梯度是使用内部二阶精确的中心差异和边界处的一阶差异或二阶精确的一侧（向前或向后）差异来计算的。
    # 因此，返回的梯度与输入数组具有相同的形状。
    y_sm_2d = np.gradient(y_sm_1d)
    # weighting function for the 2nd der:二阶权函数
    y_sm_2d_decay = (np.mean(y_sm_2d ** 2)) ** 0.5
    # 二阶权值数组平方后均值在开方
    weifunc2D = np.exp(-y_sm_2d ** 2 / 2 / y_sm_2d_decay ** 2)
    # weighting function for the 1st der:
    y_sm_1d_decay = (np.mean((y_sm_1d - np.mean(y_sm_1d)) ** 2)) ** 0.5
    weifunc1D = np.exp(-(y_sm_1d - np.mean(y_sm_1d)) ** 2 / 2 / y_sm_1d_decay ** 2)
    weifunc = weifunc1D * weifunc2D
    # exclude from screenenig the edges of the spectrum (optional)，排除光谱的边缘
    weifunc[0:zero_step_struct_el] = 1
    weifunc[-zero_step_struct_el:] = 1

    # estimate the peak height，估计峰值高度
    peakscreen_amplitude = (np.max(detrend(y)) - np.min(
        detrend(y))) / 2
    # 尚不理解：8 is good, because this is a characteristic height of a tail
    """
    detrend(y),去除光谱趋势
    """
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    D = als_lambda * D.dot(D.transpose())
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    # k = 10 * morphological_noise(y)
    #  above this height the peaks are rejected
    for i in range(_iterations):
        W.setdiag(w)
        Z = W + D
        z = spsolve(Z, w * y)
        w = als_p_weight * weifunc * np.exp(-((y - z) / peakscreen_amplitude) ** 2 / 2) * \
            (y > z) + (1 - als_p_weight) * (y < z)
    # baseline = z
    corrected_data = y - z
    return corrected_data.tolist()


# experimental_spectrum = np.genfromtxt\
#     (r'C:\Users\lixin\Desktop\测试数据\5种分类\#01\1-1200g-2-10s_1.txt')
# x = experimental_spectrum[:, 0]
# exp_y = experimental_spectrum[:, 1]
# bldp = background_substruction(exp_y)
#
# plt.figure()
# plt.plot(x, exp_y, 'k', label='exp.spec', LineWidth=1)
# plt.plot(x, exp_y - bldp, 'r', label='derpsalsa', LineWidth=1)
# plt.show()


# 1读取文件夹并合并所有文件
# 判断字符串是否为数字
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    return False


# 合并文件夹，分类
def readMerge(dataPath):
    root = dataPath
    # file_names = os.listdir(root)
    file_names = []
    # 存储所有txt文档文件名
    f_list = os.listdir(root)
    # print("&1",f_list)
    # f_list.sort(key= lambda x:int(x[-6:-4]))
    # print("&2", f_list)
    f_list.sort()
    # 排序
    # print ("**",f_list)
    for i in f_list:
        # os.path.splitext():分离文件名与扩展名
        if os.path.splitext(i)[1] == '.txt':
            # 判断是否为txt文档，
            file_names.append(i)

    file_ob_list = []
    for file_name in file_names:
        # 循环地给这520个文件名加上它前面的路径，以得到它的具体路径
        fileob = root + '/' + file_name
        # 文件夹路径加上/ 再加上具体要读的的txt的文件名就定位到了这个txt
        file_ob_list.append(fileob)
    # temp = np.loadtxt(file_ob_list[0])
    # allDataSpe = np.array(temp)[:, 0]
    # for fileNameInter in file_ob_list:
    #     temp = np.loadtxt(file_ob_list[0])
    #     allDataSpe = np.column_stack((allDataSpe, np.array(temp)[:, 1]))
    # print("11:",file_ob_list)
    firLine = np.array(pd.read_csv(file_ob_list[0], delim_whitespace=True, header=None))[0]
    # 存储首行内容
    if (is_number(firLine[0]) and is_number(firLine[1])):
        # 首行数字
        # print("首行数字")
        temp = pd.read_csv(file_ob_list[0], delim_whitespace=True, header=None)
        allDataSpe = np.array(temp)[:, 0]
        # allDataSpe = np.column_stack((allDataSpe, allDataSpe1))
        # print("**",allDataSpe.shape)
        # return 0
        # COUNT = 0
        # 用于查找编码格式错误的问题
        for fileNameInter in file_ob_list:
            # print("?",fileNameInter)
            # 打印出当前数据路径
            temp = pd.read_csv(fileNameInter, delim_whitespace=True, header=None, encoding='')
            # print("##", COUNT)
            allDataSpe = np.column_stack((allDataSpe, np.array(temp)[:, 1]))
            # COUNT = COUNT+1
            # print("**",COUNT)
            # print("**",allDataSpe.shape)
    else:
        # 首行非数字
        # print("首行非数字")
        temp = pd.read_csv(file_ob_list[0], delim_whitespace=True)
        allDataSpe = np.array(temp)[:, 0]
        for fileNameInter in file_ob_list:
            temp = pd.read_csv(fileNameInter, delim_whitespace=True)
            allDataSpe = np.column_stack((allDataSpe, np.array(temp)[:, 1]))
    # print(allDataSpe.shape)
    return np.array(allDataSpe, dtype=np.float64)

    #


# 一个文件夹下聚类，记住小翟的id标签号
# 判断字符串是否为数字
# def is_number(s):
#     try:
#         float(s)
#         return True
#     except ValueError:
#         pass
#
#     try:
#         import unicodedata
#         unicodedata.numeric(s)
#         return True
#     except (TypeError, ValueError):
#         pass
#
#     return False
# 聚类
def readMerge1(dataPath):
    root = dataPath
    # file_names = os.listdir(root)
    file_names = []
    # 存储所有txt文档文件名
    f_list = os.listdir(root)
    # print f_list
    for i in f_list:
        # os.path.splitext():分离文件名与扩展名
        if os.path.splitext(i)[1] == '.txt':
            # 判断是否为txt文档，
            file_names.append(i)

    file_ob_list = []
    idLabel = []
    # 用于存储小翟的id标签号
    for file_name in file_names:
        # 循环地给这520个文件名加上它前面的路径，以得到它的具体路径
        fileob = root + '/' + file_name

        # 识别标签，保存文件名
        if (len(file_name) < 12):
            idLabel.append(file_name[:-4])
        else:
            # 自动选择,命名规则“#001_001.txt”
            if (file_name[-12] == '#'):
                idLabel.append(file_name[-12: -4])
            else:
                idLabel.append(file_name[:-4])

        file_ob_list.append(fileob)
    # print(idLabel)
    # return 0
    firLine = np.array(pd.read_csv(file_ob_list[0], delim_whitespace=True, header=None, encoding='utf-8'))[0]
    # 存储首行内容
    if (is_number(firLine[0]) and is_number(firLine[1])):
        # 首行数字
        # print("首行数字")
        temp = pd.read_csv(file_ob_list[0], delim_whitespace=True, header=None)
        # print(np.array(temp).shape)
        # return 0
        allDataSpe = np.array(temp)[:, 0]
        '''
        # 实现单个数据截谱1
        if np.array(allDataSpe).shape[0] == 1340:
            allDataSpe = allDataSpe[110 :]
        '''
        # allDataSpe = np.column_stack((allDataSpe, allDataSpe1))
        # print(allDataSpe.shape)
        # return 0
        for fileNameInter in file_ob_list:
            temp = pd.read_csv(fileNameInter, delim_whitespace=True, header=None)
            '''
            # 实现单个数据截谱2
            temp = np.array(temp)
            print(temp.shape[0])
            if np.array(temp).shape[0] == 1340:
                temp = temp[110:,:]
            '''
            # print("1",temp.shape[0])
            allDataSpe = np.column_stack((allDataSpe, np.array(temp)[:, 1]))
    else:
        # 首行非数字
        # print("首行非数字")
        temp = pd.read_csv(file_ob_list[0], delim_whitespace=True)
        allDataSpe = np.array(temp)[:, 0]
        for fileNameInter in file_ob_list:
            temp = pd.read_csv(fileNameInter, delim_whitespace=True)
            allDataSpe = np.column_stack((allDataSpe, np.array(temp)[:, 1]))

    # data = []
    return np.array(allDataSpe, dtype=np.float64), idLabel
    # 返回ldata将list型转为numpy，方便索引


# 2.1去除宇宙射线
# def Norris(y1,n):
#     y = np.diff(y1)
#     Y = copy1.copy(y1)
#     temp = (max(y1) - min(y1)) / n
#     # list
#     for i in range(1, len(y) - 3):
#         if (abs(y[i]) > temp) :
#             if(y[i]>0):
#                 Y[i+1] = Y[i]
#             else:
#                 Y[i+1] = Y[i+2]
#     return Y
def CRR(y, filterSize, dynamicFactor):
    N1 = len(y)
    # print(N)
    crrDot = []
    for i in range(filterSize, N1 - filterSize - 1):
        # maxY = y[i - filterSize:i + filterSize + 1].max()
        minY = y[i - filterSize:i + filterSize + 1].min()
        medianY = np.median(y[i - filterSize:i + filterSize + 1])
        if ((y[i] - minY) > (medianY - minY) * dynamicFactor):
            crrDot.append(i)
    N2 = len(crrDot)
    if N2 == 0:
        return y
    else:
        for j in range(N2):
            count = 0
            while (((j + count + 1) < N2) and ((crrDot[j + count + 1] - crrDot[j + count]) == 1)):
                count = count + 1
            x1 = crrDot[j] - 1
            x2 = crrDot[j + count] + 1
            y1 = y[x1]
            y2 = y[x2]
            y[crrDot[j]] = (y2 - y1) * (crrDot[j] - x1) / (x2 - x1) + y1
        return y


# 2.2 S-G滤波
# from scipy.signal import savgol_filter

# 2.3 airPLS基线扣除

# 见上文算法

# 2.4 归一化

def MaxMinNormalization(x):
    Max = np.max(x)
    Min = np.min(x)
    x = (x - Min) / (Max - Min);
    # y=x.tolist()
    return x


# 删除文件夹下所有tif图片
def del_files(path):
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith(".tif"):
                os.remove(os.path.join(root, name))
    return 0




In [3]:
# **************************************************#################################***********************************#############################*******************************一键生成报告

def oneclickReport(dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize,
                   dynamicFactor, winSG, nSG, lambdaAirPls,
                   itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize, pcaNcomponents, pca_col1,
                   pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter, nGrad, svmTestSize, svmC, xKernel,
                   clusters_N):
    # dataPath:数据文件路径
    # startSpec:截谱起始删除波数
    # endSpec：截谱尾删除波数
    # nNorris：宇宙射线阈值
    # winSG:SG滤波窗口
    # nSG:SG滤波阶次
    # lambdaAirPls:基线校正
    # itermaxAirPls:基线校正迭代次数

    # #将传入参数写入txt
    # listTable2Name = ["dataPath", "outPath", "startSpec", "endSpec", "snrNoiseindex", "delSNRmin", "delSNRmax",
    #                   "filterSize",
    #                   "dynamicFactor", "winSG", "nSG", "lambdaAirPls", "itermaxAirPls", "pltInter", "processLinewidth",
    #                   "distanceMethod", "hcaLabelsize", "pcaNcomponents", "pca_col1", "pca_col2", "pcaSize", "pcaAlpha",
    #                   "nDim", "nPerplex", "learnRate", "nInter", "nGrad", "svmTestSize", "svmC", "xKernel",
    #                   "clusters_N"]
    # listTable2Meaning = ["输入路径", "输出路径", "截谱长度(首)", "截谱长度(尾)", "静默区：尾端长度", "信噪比最小阈值", "信噪比最大阈值",
    #                      "宇宙射线：窗口宽度", "宇宙射线：动态因子", "滤波：窗口宽度", "滤波：拟合阶次",
    #                      "基线校正：lambda", "基线校正：迭代次数", "显示光谱量(未使用)", "光谱线宽(未使用)",
    #                      "HCA距离度量方法:\r0:沃德方差最小化\r1:质心欧式距离\r2:加权分组平均\r3:UPGMA算法",
    #                      "HCA(tSNE)标号大小", "PCA主成分个数", "PCA(tSNE)选择维度", "PCA(tSNE)选择维度",
    #                      "PCA(tSNE)标签大小", "PCA(tSNE)透明度", "tSNE空间维度", "Perplex", "tSNE学习率",
    #                      "tSNE最大迭代次数", "tSNE收敛误差", "测试数据占比", "SVM错误项惩罚系数",
    #                      "SVM核函数:\r0:多项式核\r1:线性核\r2:径向基核\r3:sigmod核\r4:核矩阵", "聚类堆数"]
    # listTable2Value = [dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize,
    #                    dynamicFactor, winSG,
    #                    nSG, lambdaAirPls, itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize,
    #                    pcaNcomponents, pca_col1, pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter, nGrad,
    #                    svmTestSize, svmC, xKernel, clusters_N]
    #
    listTable2Value = [dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize,
                       dynamicFactor, winSG,
                       nSG, lambdaAirPls, itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize,
                       pcaNcomponents, pca_col1, pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter,
                       nGrad, svmTestSize, svmC, xKernel, clusters_N]
    listTable2Name = ["dataPath", "outPath", "startSpec", "endSpec", "snrNoiseindex", "delSNRmin", "delSNRmax",
                      "filterSize",
                      "dynamicFactor", "winSG", "nSG", "lambdaAirPls", "itermaxAirPls", "pltInter",
                      "processLinewidth",
                      "distanceMethod", "hcaLabelsize", "pcaNcomponents", "pca_col1", "pca_col2", "pcaSize",
                      "pcaAlpha",
                      "nDim", "nPerplex", "learnRate", "nInter", "nGrad", "svmTestSize", "svmC", "xKernel",
                      "clusters_N"]

    f = open(outPath + '/' + r'parameter.txt', "w")
    for i in range(len(listTable2Value)):
        f.write(listTable2Name[i] + ':')
        f.write(str(listTable2Value[i]) + "\n")
    f.close()

    # 定义颜色和形状，使用取余运算做循环
    # colorSpectrumColour = ['#1C1C1C','#008B8B', '#006400', '#00FFFF', '#228B22', '#90EE90', '#8B8B00', '#CD5C5C',
    #                        '#698B22', '#FFA500', '#9932CC', '#3A5FCD', '#5D478B']
    #colorSpectrumColour = ['#FF7F00', '#8B0000', '#FFFF00', '#00008B', '#006400', '#90EE90', '#698B69',
    #                       '#00FFFF', '#008B45', '#9932CC', '#3A5FCD', '#5D478B', '#1C1C1C']
    colorSpectrumColour = ['#8A9FC8', '#FB8C61', '#FFFF00', '#00008B', '#006400', '#90EE90', '#698B69',
                           '#00FFFF', '#008B45', '#9932CC', '#3A5FCD', '#5D478B', '#1C1C1C']
    
    # colorSpectrumColour = ['#90EE90', '#8B0000', '#8B008B', '#008B8B', '#00008B', '#698B69', '#FFE1FF', '#FFA500',
    #                        '#00FFFF', '#008B45', '#9932CC', '#3A5FCD', '#5D478B']
    # 光谱蓝色颜色
    colorSpectrumBlue = ['0000FF', '0000EE', '0000DD', '0000CC', '0000BB', '0000AA', '000099', '000088', '000077',
                         '000066', 'BCBCBC', 'CCCCCC', 'DCDCDC', 'ECECEC']
    # 光谱渐变灰色
    colorSpectrumGrey = ['#1C1C1C', '#2C2C2C', '3C3C3C', '4C4C4C', '5C5C5C', '6C6C6C', '7C7C7C', '8C8C8C', '9C9C9C',
                         'ACACAC', 'BCBCBC', 'CCCCCC', 'DCDCDC', 'ECECEC']
    colors = ['#1C1C1C', '#0000FF', '#FFFF00', '#7FFFD4', '#008B45']
    # markers = ["s", "o", "+", "v", "d", "x", "3", "p", ".", "*", "1","<",">",","]
    # "正方形"，"圈"，"加号"，"倒三角"，"细钻"，"×号"，"tri_left","五角"，“点”，“tri_down”,"左三角形","右三角形","像素"
    markers = ["s", '^', "o", "+", "v", "d", "x", "*", "h", "3", "p", ".", "1", "<", ">", "D"]
    # (1)□(2)△(3)○(4)＋(5)▽(6)♢(7)×(8)☆(9)“正六边形”(10)“左Y”(11)“正五边形”(12)·(13)“正Y”(14)◁(15)▷(16)◇
    files = os.listdir(dataPath)
    # 返回指定的文件夹包含的文件或文件夹的名字的列表
    # ********************************************************************************************************************************无文件夹
    if len(files) == 0:
        return 0
    # *********************************************************************************************************************************一个文件夹
    elif len(files) == 1:
        # *******************************************************************************************将该文件夹下数据合并
        dataPathOne = dataPath + "/" + files[0]
        # print("路径：",datapath1)
        dataFileOne, idLabel = readMerge1(dataPathOne)
        # print("合并后数据",dataOneFile.shape)
        # idLabel:存储映射后的id标签，用于后续返回给洪姐
        idLabelAll = copy1.copy(idLabel)
        # return 0
        # **********************************************************************************************************截谱
        dataFileOne = dataFileOne[startSpec: (dataFileOne.shape[0] - endSpec), :]

        # print("合并后数据",dataOneFile.shape)
        dataFileOne1 = copy1.copy(dataFileOne)
        # 注意：直接采用 ’ = ’ 赋值方式会使得两个变量占用同一地址
        x = dataFileOne[:, 0]
        snrFileOne = []
        # 存储拉曼光谱信噪比
        snrDelfileOne = []
        # 存储删除信噪比小于或大于阈值的光谱
        idLabelOneDel = []
        # #存储删除信噪比光谱的小翟映射ID
        idLabelOne = []
        countDelSpect = 0
        # 用于统计被删除的光谱数量
        meanSpectrumOne = []
        # 光谱均值
        stdSpectrumOne = []
        # 光谱方差

        for i in range(1, dataFileOne.shape[1]):
            snrS = 0
            # 初始化该光谱信噪比
            # *****************************************************************************************循环处理每个光谱
            # idLabel.append(i)
            # 如果不识别标签使用该语句，idLabel用ID代替
            y = dataFileOne[:, i]
            # 从第二列开始，保留了第一列坐标，后面需要去掉
            y1 = CRR(y, filterSize, dynamicFactor)
            y2 = savgol_filter(y1, winSG, nSG)
            # 2.3 airPLS
            y3 = airPLS(y2, lambdaAirPls, itermaxAirPls)
            # 有参去背景（自适应迭代重加权惩罚最小二乘，airpls）
            # y3 = background_substruction(y2)
            # 无参去背景（交替最小二乘ALS）
            # 2.4 归一化
            y4 = MaxMinNormalization(y3)
            # ********************************************************************************************信噪比计算应用
            snrSpectrumSingal = max(y4) - min(y4)
            # 信号强度
            snrSpectrumNoise = max(y4[-snrNoiseindex:]) - min(y4[-snrNoiseindex:])
            # 噪声强度
            snrS = snrSpectrumSingal / snrSpectrumNoise
            # 光谱信噪比
            if snrS < delSNRmin or snrS > delSNRmax:
                snrDelfileOne.append(i)
                # 存储删除实际标签
                idLabelOneDel.append(idLabel[i - 1])
                # # 存储删除小翟映射标签
                countDelSpect = countDelSpect + 1
                continue

            snrFileOne.append(snrS)
            # 信噪比拼接
            dataFileOne[:, i - countDelSpect] = y4
            idLabelOne.append(idLabel[i - 1])
            # 删除后剩余标签映射
        # ***************************************************************************************************************均值方差图
        figure()
        dataFileOne = dataFileOne[:, 0:(dataFileOne.shape[1] - countDelSpect)]
        # 替换并删除尾部被剔除的光谱，不然后续均值方差图会出现很宽的方差bar
        # **********************************************************************************************将信噪比保存到CSV

        snrCSV = np.array(snrFileOne).reshape(-1, 1)
        labelCSV = np.array(idLabelOne).reshape(-1, 1)
        SNRClass = np.column_stack((snrCSV, labelCSV))
        outPuthExcSNR = outPath + '/' + r'SNR(After processing).csv'
        snrList = pd.DataFrame(SNRClass, columns=["SNR", "LabelName"])
        snrList.to_csv(outPuthExcSNR)
        # *************************************************************************************************保存处理后数据
        np.savetxt(outPath + '/' + files[0] + r'_Processed.txt',
                   dataFileOne, delimiter='\t', fmt='%.2f')

        meanSpectrumOne.append(np.mean(dataFileOne[:, 1:dataFileOne.shape[1]], axis=1))
        # 按照行进行均值拼接
        stdSpectrumOne.append(np.std(dataFileOne[:, 1:dataFileOne.shape[1]], axis=1))
        # 按照行进行标准差拼接

        plt.plot(x, np.array(meanSpectrumOne).T, color="#1C1C1C",
                 lw=1,
                 label='Spectral mean')

        # plt.plot(x, np.array(meanSpectrum).T, color="white", lw=2)
        # print(np.array(x).shape)
        std1 = (np.array(meanSpectrumOne) - np.array(stdSpectrumOne)).T
        std2 = (np.array(meanSpectrumOne) + np.array(stdSpectrumOne)).T
        # 方差的上下bar
        plt.fill_between(x, std1.flatten(), std2.flatten(), color="#B5B5B5", label='Std of SNR')
        plt.legend(loc='best')
        # flatten()二维转一维
        outPathOne1 = outPath + '/' + files[0] + r'_meanStdOne.tif'
        plt.savefig(outPathOne1, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathOne1 = outPath + '/' + files[0] + r'_meanStdOne.pdf'
        plt.savefig(outPathOne1, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        # plt.show()
        plt.close()

        # **************************************************************************************************************tSNE
        figure(2)
        dataFileOne = dataFileOne[:, 1:dataFileOne.shape[1]].T

        # print(dataFileOne.shape)
        # return 0
        # 去掉首列X信息，并转置
        # 根据两个最大的主成分进行绘图

        # nDim = 2
        # #int, 空间的维度
        # nPerplex = 10.0
        # # 数据集越大，需要参数值越大，建议值位5 - 50
        # learnRate = 200
        # #float,学习率，建议取值为10.0-1000.0
        # nInter = 1000
        # # int, 最大迭代次数
        # nGrad = 1e-06
        # # float,如果梯度低于该值，则停止算法
        resultOne = TSNE(n_components=nDim, perplexity=nPerplex, early_exaggeration=12.0, learning_rate=learnRate,
                         n_iter=nInter,
                         n_iter_without_progress=300, min_grad_norm=nGrad, metric='euclidean', init='pca',
                         verbose=0, random_state=1, method='exact').fit_transform(dataFileOne)
        # random_state:int, RandomState or None, default:None，伪随机数生成器种子,不同的初始化可能会导致成本函数的局部最小值不同
        # print("***",np.array(resultOne).shape)
        # print("###",np.array(dataFileOne).shape)

        # # **************************************************************************************************************3 hca层次聚类
        #
        ySpec2 = dataFileOne

        if distanceMethod == 0:
            distM = "ward"
            # 沃德方差最小化算法
        elif distanceMethod == 1:
            distM = "centroid"
            # 质心间的欧式距离
        elif distanceMethod == 2:
            distM = "weighted"
            # 加权分组平均法
        else:
            distM = 'average'
            # UPGMA算法（非加权组平均）法

        # distanceMethod = 'ward'
        # # 层次聚类距离度量选用的方法,可选：'centroid','weighted','average'
        Z = hierarchy.linkage(ySpec2, method=distM, metric='euclidean')
        # 执行层次/聚集聚类

        hcaColorThreshold = 10

        # color_threshold= 8 设置颜色高度阈值，用于控制决策树和PCA的颜色分类
        # hcaLabelsize = 6
        # #标号大小
        # cluster.hierarchy.cut_tree(Z, n_clusters=filesNumInter)
        # 设置聚类个数n_clusters

        hierarchy.dendrogram(Z, color_threshold=hcaColorThreshold, labels=pd.DataFrame(ySpec2).index + 1)
        # 将分层聚类绘制为树状图
        # hierarchy.dendrogram(Z, labels=pd.DataFrame(ySpec).index + 1)
        # label = cluster.hierarchy.cut_tree(Z, height = hcaColorThreshold)
        hacLeaves = hierarchy.leaves_list(Z) + 1
        # 返回叶子节点
        plt.tick_params(labelsize=hcaLabelsize)
        # 调整刻度尺寸
        plt.title('HCA')
        # plt.show()
        outPath7 = outPath + '/' + r'_HCAcluster.tif'
        plt.savefig(outPath7, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath7 = outPath + '/' + r'_HCAcluster.pdf'
        plt.savefig(outPath7, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        # plt.show()
        plt.close()
        # # *******************************************************************************************保存层次聚类叶子节点
        f = open(outPath + '/' + r'_HCAclusterLeaves.txt', "w")
        f.write("层次聚类树状图叶子节点（从左到右排列）：")
        f.write('\n')
        f.write(str(hacLeaves))
        f.close()

        # ***************************************************************************************************HCA聚类评估
        f = open(outPath + '/' + r'_HCA-evaluation(聚类评估算法).txt', "w", encoding='utf-8')
        f.write("1.轮廓系数法：\n")
        f.write("  轮廓系数（Silhouette Coefficient），是聚类效果好坏的一种评价方式,轮廓系数计算方式s=(b-a)/max(a,b)，"
                "a：样本与同一簇中所有其他点之间的平均距离，b：样本与下一个最近的簇中所有其他点之间的平均距离。"
                "最佳值为1，最差值为-1。接近0的值表示重叠的群集。负值通常表示样本已分配给错误的聚类，因为不同的聚类更为相似。"
                "-1表示存在错误聚类，0表示有交叉的簇，1表示高密度聚类\n")
        f.write("2.Calinski-Harabaz（CH）：\n")
        f.write("  Calinski-Harabaz（CH）,CH指数S被定义为簇间离散与簇内离散的比率，"
                "当簇内数据的协方差越小，簇间的协方差越大，Calinski-Harabasz分数越高，该分值越大说明聚类效果越好。")
        f.close()

        # *************************************************************************************************轮廓系数K初值
        K = 10
        if dataFileOne.shape[1] < K:
            clustersK = range(2, dataFileOne.shape[1])
        else:
            clustersK = range(2, K)
        # clustersK = range(2, 10)
        sc_scores = []
        # 轮廓系数值
        CHindexAgglomerativeClustering = []
        # Calinski-Harabasz Index
        for t in clustersK:
            kmeans_model = AgglomerativeClustering(n_clusters=t, affinity='euclidean', memory=None, connectivity=None,
                                                   compute_full_tree='auto', linkage='single').fit(dataFileOne)
            sc_score = silhouette_score(dataFileOne, kmeans_model.labels_, metric='euclidean')
            sc_scores.append(sc_score)
            # SC系数(Silhouette Cofficient)轮廓系数法
            '''
            轮廓系数计算方式是，a：样本与同一类别中所有其他点之间的平均距离，b：样本与下一个最近的簇中所有其他点之间的平均距离。
            单个样本的Silhouette系数s为：(b-a)/max(a,b),此函数返回所有样本的平均轮廓系数
            仅当标签数为2 <= n_labels <= n_samples-1时，才定义轮廓系数。
            最佳值为1，最差值为-1。接近0的值表示重叠的群集。负值通常表示样本已分配给错误的聚类，因为不同的聚类更为相​​似。
            -1表示存在错误聚类，0表示有交叉的簇，1表示高密度聚类。
            '''
            CHindexAgglomerativeClustering.append(
                metrics.calinski_harabasz_score(dataFileOne, kmeans_model.labels_))
                #metrics.calinski_harabasz_score(X, y_pred)
            # metrics.calinski_harabasz_score
            # metrics.calinski_harabaz_score（注意区分两个包的区别）
            '''
            Calinski-Harabaz的分数S被定义为簇间离散与簇内离散的比率，该分值越大说明聚类效果越好。
            '''
        # kmeans_model = KMeans(n_clusters=3).fit(resultOne)
        # sc_score = silhouette_score(resultOne, kmeans_model.labels_, metric='euclidean')
        # print("##",sc_score)
        # return 0
        fig, ax1 = plt.subplots(1, 1)  # 做1*1个子图，等价于 " fig, ax1 = plt.subplot() "
        ax2 = ax1.twinx()  # 让2个子图的x轴一样，同时创建副坐标轴。
        ax1.set_xlabel('Numbers of clusters(HCA)')
        color1 = '#228B22'
        ax1.set_ylabel('silhouette coefficient score', color=color1)
        ax1.plot(clustersK, sc_scores, marker='o', color=color1)
        ax1.tick_params(axis='y', labelcolor=color1)

        color2 = '#8B6914'
        ax2.set_ylabel('Calinski-Harabasz', color=color2)
        ax2.plot(clustersK, CHindexAgglomerativeClustering, color=color2)
        ax2.tick_params(axis='y', labelcolor=color2)

        fig.tight_layout()
        outPathHCA1 = outPath + '/' + r'_HCA-evaluation.tif'
        plt.savefig(outPathHCA1, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathHCA2 = outPath + '/' + r'_HCA-evaluation.pdf'
        plt.savefig(outPathHCA2, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ****************************************************************************************AgglomerativeClustering

        # clusters_N = 2
        clf = AgglomerativeClustering(n_clusters=clusters_N, affinity='euclidean', memory=None, connectivity=None,
                                      compute_full_tree='auto', linkage='single')
        # s = clf.fit(resultOne)
        try:
            s = clf.fit(dataFileOne)
        except:
            s = clf.fit(dataFileOne)
        #
        # clf = SpectralClustering(affinity='nearest_neighbors', n_clusters=clusters_N, n_neighbors=11).fit_predict(resultOne)
        #
        # print('KMeans参数为：', s)

        # pcaSize = 30
        # #pca标签大小
        # pcaAlpha = 1
        # #pca透明度
        for i in range(resultOne.shape[0]):
            plt.scatter(resultOne[i, pca_col1], resultOne[i, pca_col2],
                        c=colorSpectrumColour[clf.labels_[i] % len(colorSpectrumColour)],
                        marker=markers[clf.labels_[i] % len(markers)], s=pcaSize, alpha=pcaAlpha, edgecolor='k')

        # # 画出聚类中心
        # centerS = clf.cluster_centers_
        # for i in range(clusters_N):
        #     plt.scatter(centerS[i][0], centerS[i][1], c="red", s=pcaSize + 6)

        # **************************************************************************************************************根据聚类结果，重新构建各值list
        labelOneResult = []
        # 返回标签给程序
        labelOneResultL = []
        # 返回标签给小翟
        clusterSnr = []
        # 存储聚类后的信噪比二维list
        clusterStd = []
        # 存储聚类后光谱标准差二维list
        clusterData = []
        # 存储聚类后光谱数据二维list
        hcaDistance = []
        # 存储聚类距离，保存聚类中心到各点距离
        spectralDistance = []
        # 存储谱聚类距离，保存聚类中心到各点距离
        centerS_HCA = []
        # 保存聚类中心，其实是计算kmeansDistance的均值（平面欧式距离）

        # 创建二维列表
        for i in range(clusters_N):
            labelOneResult.append([])
            labelOneResultL.append([])
            clusterSnr.append([])
            # clusterStd.append([])
            clusterData.append([])
            hcaDistance.append([])

        # 根据聚类结果重新组织矩阵，每个子list存放一类
        for i in range(clusters_N):
            for j in range(len(clf.labels_)):
                if (clf.labels_[j] == i):
                    labelOneResult[i].append(clf.labels_[j])
                    labelOneResultL[i].append(idLabelOne[j])
                    clusterSnr[i].append(snrFileOne[j])
                    clusterData[i].append(dataFileOne[j].T)
                    hcaDistance[i].append([resultOne[j, 0], resultOne[j, 1]])
                    # hca和谱聚类与kmeans不同，多一个计算过程
            centerS_HCA.append(np.array(hcaDistance[i]).mean(axis=0).tolist())
            # print(np.array(kmeansDistance[0]).mean(axis = 0)),列均值，求聚类散点的均值（均值即聚类中心，欧式距离）

        # # 画出聚类中心
        # centerS = clf.cluster_centers_
        centerS_HCA = np.array(centerS_HCA)
        for i in range(clusters_N):
            plt.scatter(centerS_HCA[i][0], centerS_HCA[i][1], c="red", s=pcaSize + 6)

        # plt.legend(loc=0, ncol=1)
        plt.xlabel(u"tSNE 1")
        plt.ylabel(u"tSNE 2")
        plt.title('HCA-tSNE')
        # plt.show()
        outPath9 = outPath + '/' + r'_HCA-tSNE.tif'
        plt.savefig(outPath9, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath9 = outPath + '/' + r'_HCA-tSNE.pdf'
        plt.savefig(outPath9, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ***************************************************************************************************************计算聚类中心到聚类数据距离
        # 存储每类均谱
        clusterMeanS = []
        clusterStdS = []
        std1 = []
        std2 = []
        f = open(outPath + '/' + r'_HCA-tSNE(Clustering evaluation).txt', "w")
        for i in range(clusters_N):
            clusterMeanS.append(np.mean(clusterData[i], axis=0))
            clusterStdS.append(np.std(clusterData[i], axis=0))
            std1.append((np.array(clusterMeanS[i]) - np.array(clusterStdS[i])))
            std2.append((np.array(clusterMeanS[i]) + np.array(clusterStdS[i])))
            # print("kmeans1:",np.mean(cdist([centerS_HCA[i].tolist()],kmeansDistance[i] , metric='euclidean')))
            # print("center1:",
            #       (np.sum(cdist([centerS_HCA[i].tolist()], centerS_HCA, metric='euclidean'))) / (len(centerS_HCA) - 1))
            f.write("HCA聚类结果中，第" + str(i + 1) + "堆聚类中心到该堆内所有点欧式距离均值：" + str("%.2f" % (np.mean(
                cdist([centerS_HCA[i].tolist()], hcaDistance[i], metric='euclidean')))) + "\n")

            f.write("HCA聚类结果中，第" + str(i + 1) + "堆聚类中心到其他所有聚类中心欧式距离均值：" + str(
                "%.2f" % ((np.sum(cdist([centerS_HCA[i].tolist()], centerS_HCA, metric='euclidean'))) / (
                        len(centerS_HCA) - 1))) + "\n")
            # (len(centerS)-1):聚类中心点个数-1，当前点到其他聚类中心的均值
            # cdist([centerS[i].tolist()], centerS, metric='euclidean')，点到其他所有点距离
            # np.sum：距离求和，均值需要除以n-1，即：len(centerS)-1
        # f.write("\n\n标签顺序：(1)□(2)◇(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·(13)正Y(14)◁(15)▷(16)△(也可按箱线图或均值方差罗列图颜色对应)")
        f.write(
            "\n\n标签顺序：(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·"
            "(13)正Y(14)左三角形(15)右三角形(16)◇(也可按箱线图或均值方差罗列图颜色对应)")
        f.close()

        # **************************************************************************************************************保存HCA聚类均值
        hcaclusterMeanS = np.column_stack((x, np.array(clusterMeanS).T))
        np.savetxt(outPath + '/' + r'_HCA-means.txt',
                   hcaclusterMeanS, delimiter='\t', fmt='%.2f')

        # **************************************************************************************************************根据聚类结果，画均值方差图
        figure(figsize=(8, 12))
        for interI in range(clusters_N):
            interIhight = 1
            # print("1:",np.array(x).shape)
            # print("2:",np.array(clusterData[interI]).shape)
            # return 0
            plt.plot(x, (clusterMeanS[interI] + interIhight * interI),
                     color=colorSpectrumColour[interI % len(colorSpectrumColour)], lw=1)
            plt.fill_between(x, (std1[interI]) + interIhight * interI, (std2[interI]) + interIhight * interI,
                             color="#B5B5B5")
        plt.title('MeanStd column chart', fontsize=12)
        # plt.show()

        # plt.legend(loc = 'best')
        outPath4 = outPath + '/' + r'_HCA-tSNEmeanStd.tif'
        plt.savefig(outPath4, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath4 = outPath + '/' + r'_HCA-tSNEmeanStd.pdf'
        plt.savefig(outPath4, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #
        # for i in range(clusters_N):
        #     meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #     # 按照行进行均值拼接
        #     stdSpectrum.append(np.std(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))

        # **************************************************************************************************************根据聚类结果，画箱线图
        # figure()
        dfLists = []
        # 存储以文件为单位信噪比字典
        for interI in range(clusters_N):
            dfLists.append(pd.DataFrame({"cluster:" + str(interI + 1): clusterSnr[interI]}))
        dfData = pd.concat(dfLists, axis=1)

        f = dfData.boxplot(sym='+', patch_artist=True, return_type='dict')

        for box, c in zip(f['boxes'], colorSpectrumColour):
            # 箱体边框颜色
            box.set(color=c, linewidth=2)
            # 箱体内部填充颜色
            box.set(facecolor=c)

        plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.8)

        outPath41 = outPath + '/' + r'_HCA-tSNEbox.tif'
        plt.savefig(outPath41, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath41 = outPath + '/' + r'_HCA-tSNEbox.pdf'
        plt.savefig(outPath41, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ***************************************************************************************************************保存txt聚类结果

        f = open(outPath + '/' + r'_HCA-tSNE.txt', "w")
        for i in range(clusters_N):
            f.write(str(i + 1) + "\t")
            for j in range(len(labelOneResultL[i])):
                # f.write(str((labelOneResultL[i])[j]) + ",")
                f.write(str((labelOneResultL[i])[j]))
                if j < len(labelOneResultL[i]) - 1:
                    f.write(",")
            f.write('\n')

        f.close()

        # ************************************************************************************************************************************kemans聚类
        # ****************************************************************************************基于轮廓系数的kmeans聚类评估
        sc_scores = []
        # 轮廓系数值
        CHindexKmeans = []
        # Calinski-Harabasz Index
        for t in clustersK:
            kmeans_model = KMeans(n_clusters=t).fit(dataFileOne)
            sc_score = silhouette_score(dataFileOne, kmeans_model.labels_, metric='euclidean')
            sc_scores.append(sc_score)
            # SC系数(Silhouette Cofficient)轮廓系数法
            '''
            轮廓系数计算方式是，a：样本与同一类别中所有其他点之间的平均距离，b：样本与下一个最近的簇中所有其他点之间的平均距离。
            单个样本的Silhouette系数s为：(b-a)/max(a,b),此函数返回所有样本的平均轮廓系数
            仅当标签数为2 <= n_labels <= n_samples-1时，才定义轮廓系数。
            最佳值为1，最差值为-1。接近0的值表示重叠的群集。负值通常表示样本已分配给错误的聚类，因为不同的聚类更为相​​似。
            -1表示存在错误聚类，0表示有交叉的簇，1表示高密度聚类。
            '''
            labelsKm = metrics.calinski_harabasz_score(dataFileOne, kmeans_model.labels_)
            CHindexKmeans.append(labelsKm)

            '''
            Calinski-Harabaz的分数S被定义为簇间离散与簇内离散的比率，该分值越大说明聚类效果越好。
            '''
        # kmeans_model = KMeans(n_clusters=3).fit(resultOne)
        # sc_score = silhouette_score(resultOne, kmeans_model.labels_, metric='euclidean')
        # print("##",sc_score)
        # return 0
        fig, ax1 = plt.subplots(1, 1)  # 做1*1个子图，等价于 " fig, ax1 = plt.subplot() "
        ax2 = ax1.twinx()  # 让2个子图的x轴一样，同时创建副坐标轴。
        ax1.set_xlabel('Numbers of clusters(Kmeans)')
        color1 = '#228B22'
        ax1.set_ylabel('silhouette coefficient score', color=color1)
        ax1.plot(clustersK, sc_scores, marker='o', color=color1)
        ax1.tick_params(axis='y', labelcolor=color1)

        color2 = '#8B6914'
        ax2.set_ylabel('Calinski-Harabasz', color=color2)
        ax2.plot(clustersK, CHindexKmeans, color=color2)
        ax2.tick_params(axis='y', labelcolor=color2)

        fig.tight_layout()
        outPathKm1 = outPath + '/' + r'_kMeans-evaluation.tif'
        plt.savefig(outPathKm1, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathKm2 = outPath + '/' + r'_kMeans-evaluation.pdf'
        plt.savefig(outPathKm2, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # *****************************************************************************************************kemans聚类
        # clusters_N = 2
        clf = KMeans(n_clusters=clusters_N)
        # clf = AgglomerativeClustering(n_clusters=clusters_N, affinity='euclidean', memory=None, connectivity=None,
        #                               compute_full_tree='auto', linkage='single')
        # s = clf.fit(resultOne)
        try:
            s = clf.fit(dataFileOne)
        except:
            s = clf.fit(dataFileOne)
        # pcaSize = 30
        # #pca标签大小
        # pcaAlpha = 1
        # #pca透明度
        for i in range(resultOne.shape[0]):
            plt.scatter(resultOne[i, pca_col1], resultOne[i, pca_col2],
                        c=colorSpectrumColour[clf.labels_[i] % len(colorSpectrumColour)],
                        marker=markers[clf.labels_[i] % len(markers)], s=pcaSize, alpha=pcaAlpha, edgecolor='k')

        # 画出聚类中心
        centerS = clf.cluster_centers_
        # print("centerS:",centerS.shape)
        # print("centerS:",centerS[0])
        # return 0
        for i in range(clusters_N):
            plt.scatter(centerS[i][0], centerS[i][1], c="red", s=pcaSize + 6)

        # for tsneI in range(3):
        #     tsneCount2 = tsneCount2 + pcaCount[tsneI]
        #     plt.scatter(result2[tsneCount1:tsneCount2, pca_col1], result2[tsneCount1:tsneCount2, pca_col2],
        #                 c=colorSpectrumColour[tsneI % len(colorSpectrumColour)], marker=markers[tsneI % len(markers)],
        #                 s=pcaSize,
        #                 alpha=pcaAlpha, edgecolor='k', label=files[tsneI])
        #     tsneCount1 = tsneCount1 + pcaCount[tsneI]

        # plt.legend(loc=0, ncol=1)
        plt.xlabel(u"tSNE 1")
        plt.ylabel(u"tSNE 2")
        plt.title('kMeans-tSNE')
        # plt.show()
        outPath9 = outPath + '/' + r'_kMeans-TSNE.tif'
        plt.savefig(outPath9, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath9 = outPath + '/' + r'_kMeans-TSNE.pdf'
        plt.savefig(outPath9, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # **************************************************************************************************************根据聚类结果，重新构建各值list
        labelOneResult = []
        # 返回标签给程序
        labelOneResultL = []
        # 返回标签给小翟
        clusterSnr = []
        # 存储聚类后的信噪比二维list
        clusterStd = []
        # 存储聚类后光谱标准差二维list
        clusterData = []
        # 存储聚类后光谱数据二维list
        kmeansDistance = []
        # 存储聚类距离，保存聚类中心到各点距离

        # 创建二维列表
        for i in range(clusters_N):
            labelOneResult.append([])
            labelOneResultL.append([])
            clusterSnr.append([])
            # clusterStd.append([])
            clusterData.append([])
            kmeansDistance.append([])

        # 根据聚类结果重新组织矩阵，每个子list存放一类
        for i in range(clusters_N):
            for j in range(len(clf.labels_)):
                if (clf.labels_[j] == i):
                    labelOneResult[i].append(clf.labels_[j])
                    labelOneResultL[i].append(idLabelOne[j])
                    clusterSnr[i].append(snrFileOne[j])
                    clusterData[i].append(dataFileOne[j].T)
                    kmeansDistance[i].append([resultOne[j, 0], resultOne[j, 1]])

        # ***************************************************************************************************************计算聚类中心到聚类数据距离
        # 存储每类均谱
        clusterMeanS = []
        clusterStdS = []
        std1 = []
        std2 = []
        f = open(outPath + '/' + r'_kMeans-tSNE(Clustering evaluation).txt', "w")
        for i in range(clusters_N):
            clusterMeanS.append(np.mean(clusterData[i], axis=0))
            clusterStdS.append(np.std(clusterData[i], axis=0))
            std1.append((np.array(clusterMeanS[i]) - np.array(clusterStdS[i])))
            std2.append((np.array(clusterMeanS[i]) + np.array(clusterStdS[i])))
            # print("kmeans1:",np.mean(cdist([centerS_HCA[i].tolist()],kmeansDistance[i] , metric='euclidean')))
            # print("center1:",
            #       (np.sum(cdist([centerS_HCA[i].tolist()], centerS_HCA, metric='euclidean'))) / (len(centerS_HCA) - 1))

            # f.write("Kmeans聚类结果中，第" + str(i + 1) + "堆聚类中心到该堆内所有点欧式距离均值：" + str(
            #     "%.2f" % (np.mean(cdist([centerS[i].tolist()], kmeansDistance[i], metric='euclidean')))) + "\n")
            # f.write("Kmeans聚类结果中，第" + str(i + 1) + "堆聚类中心到其他所有聚类中心欧式距离均值：" + str(
            #     "%.2f" % ((np.sum(cdist([centerS[i].tolist()], centerS, metric='euclidean'))) / (
            #             len(centerS) - 1))) + "\n")
            # (len(centerS)-1):聚类中心点个数-1，当前点到其他聚类中心的均值
            # cdist([centerS[i].tolist()], centerS, metric='euclidean')，点到其他所有点距离
            # np.sum：距离求和，均值需要除以n-1，即：len(centerS)-1
        # f.write("\n\n标签顺序：(1)□(2)◇(3)○(4)＋(5)▽(6)♢(7)×(8)☆(9)“正六边形”(10)“左Y”(11)“正五边形”
        # (12)·(13)“正Y”(14)◁(15)▷(16)△(也可按箱线图或均值方差罗列图颜色对应)")
        f.write(
            "\n\n标签顺序：(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·"
            "(13)正Y(14)左三角形(15)右三角形(16)◇(也可按箱线图或均值方差罗列图颜色对应)")
        f.close()
        # **************************************************************************************************************保存谱kmeans聚类均值
        hcaclusterMeanS = np.column_stack((x, np.array(clusterMeanS).T))
        np.savetxt(outPath + '/' + r'_kMeans-means.txt',
                   hcaclusterMeanS, delimiter='\t', fmt='%.2f')
        # **************************************************************************************************************根据聚类结果，画均值方差图
        figure(figsize=(8, 12))
        for interI in range(clusters_N):
            interIhight = 0.6
            # print("1:",np.array(x).shape)
            # print("2:",np.array(clusterData[interI]).shape)
            # return 0
            plt.plot(x, (clusterMeanS[interI] + interIhight * interI),
                     color=colorSpectrumColour[interI % len(colorSpectrumColour)], lw=1)
            plt.fill_between(x, (std1[interI]) + interIhight * interI, (std2[interI]) + interIhight * interI,
                             color="#B5B5B5")
        plt.title('MeanStd column chart', fontsize=12)
        # plt.show()

        # plt.legend(loc = 'best')
        outPath4 = outPath + '/' + r'_kMeans-tSNEmeanStd.tif'
        plt.savefig(outPath4, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath4 = outPath + '/' + r'_kMeans-tSNEmeanStd.pdf'
        plt.savefig(outPath4, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #
        # for i in range(clusters_N):
        #     meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #     # 按照行进行均值拼接
        #     stdSpectrum.append(np.std(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))

        # **************************************************************************************************************根据聚类结果，画箱线图
        # figure()
        dfLists = []
        # 存储以文件为单位信噪比字典
        for interI in range(clusters_N):
            dfLists.append(pd.DataFrame({"cluster:" + str(interI + 1): clusterSnr[interI]}))
        dfData = pd.concat(dfLists, axis=1)

        f = dfData.boxplot(sym='+', patch_artist=True, return_type='dict')

        for box, c in zip(f['boxes'], colorSpectrumColour):
            # 箱体边框颜色
            box.set(color=c, linewidth=2)
            # 箱体内部填充颜色
            box.set(facecolor=c)

        plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.8)

        outPath41 = outPath + '/' + r'_kMeans-tSNEbox.tif'
        plt.savefig(outPath41, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath41 = outPath + '/' + r'_kMeans-tSNEbox.pdf'
        plt.savefig(outPath41, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ***************************************************************************************************************保存txt聚类结果

        f = open(outPath + '/' + r'_kMeans-tSNE.txt', "w")
        for i in range(clusters_N):
            f.write(str(i + 1) + "\t")
            for j in range(len(labelOneResultL[i])):
                # f.write(str((labelOneResultL[i])[j]) + ",")
                f.write(str((labelOneResultL[i])[j]))
                if j < len(labelOneResultL[i]) - 1:
                    f.write(",")
            f.write('\n')

        f.close()

        # return 0

        # # *************************************************************************************************************再次聚类，最大堆聚类
        # temp = []
        # for i in range(clusters_N):
        #     temp.append(np.array(clusterData[i]).shape[0])
        # temp = np.array(temp)
        # # temp = np.array([np.array(clusterData[0]).shape[0],np.array(clusterData[1]).shape[0],
        # #         np.array(clusterData[2]).shape[0],np.array(clusterData[3]).shape[0],
        # #         np.array(clusterData[4]).shape[0]])
        #
        # max_index = np.argmax(temp)
        #
        # # print("00:", labelOneResultL[max_index])
        #
        # labelOneResultLMax = labelOneResultL[max_index]
        # # 存放最大堆标签
        #
        # resultOne1 = TSNE(n_components=nDim, perplexity=nPerplex, early_exaggeration=12.0, learning_rate=learnRate,
        #                   n_iter=nInter,
        #                   n_iter_without_progress=300, min_grad_norm=nGrad, metric='euclidean', init='pca',
        #                   verbose=0, random_state=None, method='exact').fit_transform(clusterData[max_index])
        #
        # clusters_N1 = 3
        #
        # clf1 = KMeans(n_clusters=clusters_N1, max_iter=300, tol=1e-3, init='random', n_init=20,
        #               precompute_distances='auto', algorithm='auto')
        # clf1.fit(resultOne1)
        #
        # for i in range(resultOne1.shape[0]):
        #     plt.scatter(resultOne1[i, pca_col1], resultOne1[i, pca_col2],
        #                 c=colorSpectrumColour[clf1.labels_[i] % len(colorSpectrumColour)],
        #                 marker=markers[clf1.labels_[i] % len(markers)], s=pcaSize, alpha=pcaAlpha, edgecolor='k')
        #
        # # 画出聚类中心
        # centerS = clf1.cluster_centers_
        # for i in range(clusters_N1):
        #     plt.scatter(centerS[i][0], centerS[i][1], c="red", s=pcaSize + 6)
        # # plt.scatter(centerS[1][0], centerS[1][1], c="red", s=pcaSize + 6)
        # # plt.scatter(centerS[2][0], centerS[2][1], c="red", s=pcaSize + 6)
        #
        # # plt.legend(loc=0, ncol=1)
        # plt.xlabel(u"tSNE 1")
        # plt.ylabel(u"tSNE 2")
        # plt.title('kMeans-tSNE')
        # # plt.show()
        # outPath5 = outPath + '\\' + r'_kMeans-TSNE(max).tif'
        # plt.savefig(outPath5, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        # plt.close()
        # # **************************************************************************************************************再次聚类结果，重新构建各值list
        # labelOneResult2 = []
        # # 返回标签给程序
        # labelOneResultL2 = []
        # # 返回标签给小翟
        # clusterSnr2 = []
        # clusterData2 = []
        # # 创建二维列表
        # for i in range(clusters_N1):
        #     labelOneResult2.append([])
        #     labelOneResultL2.append([])
        #     clusterSnr2.append([])
        #     # clusterStd2.append([])
        #     clusterData2.append([])
        # # 根据聚类结果重新组织矩阵，每个子list存放一类
        # for i in range(clusters_N1):
        #     for j in range(len(clf1.labels_)):
        #         if (clf1.labels_[j] == i):
        #             labelOneResult2[i].append(clf1.labels_[j])
        #             labelOneResultL2[i].append(labelOneResultLMax[j])
        #             clusterSnr2[i].append(snrFileOne[j])
        #             clusterData2[i].append(dataFileOne[j].T)
        #
        # # 存储每类均谱
        # clusterMeanS2 = []
        # clusterStdS2 = []
        # Std1 = []
        # Std2 = []
        # for i in range(clusters_N1):
        #     clusterMeanS2.append(np.mean(clusterData2[i], axis=0))
        #     clusterStdS2.append(np.std(clusterData2[i], axis=0))
        #     Std1.append((np.array(clusterMeanS2[i]) - np.array(clusterStdS2[i])))
        #     Std2.append((np.array(clusterMeanS2[i]) + np.array(clusterStdS2[i])))
        #
        # # **************************************************************************************************************根据max聚类结果，再次画均值方差图
        #
        # for interI in range(clusters_N1):
        #     interIhight = 0.5
        #     # print("1:",np.array(x).shape)
        #     # print("2:",np.array(clusterData[interI]).shape)
        #     # return 0
        #     plt.plot(x, (clusterMeanS2[interI] + interIhight * interI),
        #              color=colorSpectrumColour[interI % len(colorSpectrumColour)], lw=1)
        #     plt.fill_between(x, (Std1[interI]) + interIhight * interI, (Std2[interI]) + interIhight * interI,
        #                      color="#B5B5B5")
        # plt.title('MeanStd column chart', fontsize=14)
        # # plt.show()
        #
        # # plt.legend(loc = 'best')
        # outPath5 = outPath + '\\' + r'_kMeans-TSNE(max)meanStd.tif'
        # plt.savefig(outPath5, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        # # plt.show()
        # plt.close()
        #
        # # # **************************************************************************************************************根据max聚类结果，再次画箱线图
        # # # figure()
        # dfLists2 = []
        # # 存储以文件为单位信噪比字典
        # for interI in range(clusters_N1):
        #     dfLists2.append(pd.DataFrame({"cluster(max):" + str(interI + 1): clusterSnr2[interI]}))
        # dfData2 = pd.concat(dfLists2, axis=1)
        #
        # f2 = dfData2.boxplot(sym='+', patch_artist=True, return_type='dict')
        #
        # for box, c in zip(f2['boxes'], colorSpectrumColour):
        #     # 箱体边框颜色
        #     box.set(color=c, linewidth=2)
        #     # 箱体内部填充颜色
        #     box.set(facecolor=c)
        #
        # plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.8)
        #
        # outPath51 = outPath + '\\' + r'_kMeans-TSNE(max)box.tif'
        # plt.savefig(outPath51, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        # # plt.show()
        # plt.close()
        #
        # # ***************************************************************************************************************再次max聚类保存txt结果
        #
        # f = open(outPath + '\\' + r'_kMeans-TSNE(max).txt', "w")
        # for i in range(clusters_N1):
        #     f.write(str(i + 1) + "\t")
        #     for j in range(len(labelOneResultL2[i])):
        #         # f.write(str((labelOneResultL[i])[j]) + ",")
        #         f.write(str((labelOneResultL2[i])[j]))
        #         if j < len(labelOneResultL2[i]) - 1:
        #             f.write(",")
        #     f.write('\n')
        #
        # f.close()

        # ************************************************************************************************************************************谱聚类
        # ****************************************************************************************基于轮廓系数的谱聚类评估
        # clustersK = range(2, 10)
        sc_scores = []
        # 轮廓系数值
        CHindexSpectralClustering = []
        # Calinski-Harabasz Index
        for t in clustersK:
            SpectralClustering_model = SpectralClustering(affinity='nearest_neighbors', n_clusters=t,
                                                          n_neighbors=11).fit(resultOne)
            sc_score = silhouette_score(resultOne, SpectralClustering_model.labels_, metric='euclidean')
            sc_scores.append(sc_score)
            # SC系数(Silhouette Cofficient)轮廓系数法
            '''
            轮廓系数计算方式是，a：样本与同一类别中所有其他点之间的平均距离，b：样本与下一个最近的簇中所有其他点之间的平均距离。
            单个样本的Silhouette系数s为：(b-a)/max(a,b),此函数返回所有样本的平均轮廓系数
            仅当标签数为2 <= n_labels <= n_samples-1时，才定义轮廓系数。
            最佳值为1，最差值为-1。接近0的值表示重叠的群集。负值通常表示样本已分配给错误的聚类，因为不同的聚类更为相​​似。
            -1表示存在错误聚类，0表示有交叉的簇，1表示高密度聚类。            
            '''
            CHindexSpectralClustering.append(
                metrics.calinski_harabasz_score(resultOne, SpectralClustering_model.labels_))

            '''
            Calinski-Harabaz的分数S被定义为簇间离散与簇内离散的比率，该分值越大说明聚类效果越好。
            '''
        # kmeans_model = KMeans(n_clusters=3).fit(resultOne)
        # sc_score = silhouette_score(resultOne, kmeans_model.labels_, metric='euclidean')
        # print("##",sc_score)
        # return 0
        fig, ax1 = plt.subplots(1, 1)  # 做1*1个子图，等价于 " fig, ax1 = plt.subplot() "
        ax2 = ax1.twinx()  # 让2个子图的x轴一样，同时创建副坐标轴。
        ax1.set_xlabel('Numbers of clusters(SpectralClustering)')
        color1 = '#228B22'
        ax1.set_ylabel('silhouette coefficient score', color=color1)
        ax1.plot(clustersK, sc_scores, marker='o', color=color1)
        ax1.tick_params(axis='y', labelcolor=color1)

        color2 = '#8B6914'
        ax2.set_ylabel('Calinski-Harabasz', color=color2)
        ax2.plot(clustersK, CHindexSpectralClustering, color=color2)
        ax2.tick_params(axis='y', labelcolor=color2)

        fig.tight_layout()
        outPathSC2 = outPath + '/' + r'_SClustering-evaluation.tif'
        plt.savefig(outPathSC2, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathSC21 = outPath + '/' + r'_SClustering-evaluation.pdf'
        plt.savefig(outPathSC21, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # *********************************************************************************************************谱聚类
        clf = SpectralClustering(affinity='nearest_neighbors', n_clusters=clusters_N, n_neighbors=11)
        try:
            s = clf.fit_predict(resultOne)
        except:
            s = clf.fit_predict(resultOne)
        # s = clf.fit_predict(resultOne)
        for i in range(resultOne.shape[0]):
            plt.scatter(resultOne[i, pca_col1], resultOne[i, pca_col2],
                        c=colorSpectrumColour[clf.labels_[i] % len(colorSpectrumColour)],
                        marker=markers[clf.labels_[i] % len(markers)], s=pcaSize, alpha=pcaAlpha, edgecolor='k')

        # for tsneI in range(3):
        #     tsneCount2 = tsneCount2 + pcaCount[tsneI]
        #     plt.scatter(result2[tsneCount1:tsneCount2, pca_col1], result2[tsneCount1:tsneCount2, pca_col2],
        #                 c=colorSpectrumColour[tsneI % len(colorSpectrumColour)], marker=markers[tsneI % len(markers)],
        #                 s=pcaSize,
        #                 alpha=pcaAlpha, edgecolor='k', label=files[tsneI])
        #     tsneCount1 = tsneCount1 + pcaCount[tsneI]

        # **************************************************************************************************************根据聚类结果，重新构建各值list
        labelOneResult = []
        # 返回标签给程序
        labelOneResultL = []
        # 返回标签给小翟
        clusterSnr = []
        # 存储聚类后的信噪比二维list
        clusterStd = []
        # 存储聚类后光谱标准差二维list
        clusterData = []
        # 存储聚类后光谱数据二维list
        spectralDistance = []
        # 存储谱聚类距离，保存聚类中心到各点距离
        centerS_SC = []
        # 保存聚类中心，其实是计算kmeansDistance的均值（平面欧式距离）

        # 创建二维列表
        for i in range(clusters_N):
            labelOneResult.append([])
            labelOneResultL.append([])
            clusterSnr.append([])
            # clusterStd.append([])
            clusterData.append([])
            spectralDistance.append([])
        # 根据聚类结果重新组织矩阵，每个子list存放一类
        for i in range(clusters_N):
            for j in range(len(clf.labels_)):
                if (clf.labels_[j] == i):
                    labelOneResult[i].append(clf.labels_[j])
                    labelOneResultL[i].append(idLabelOne[j])
                    clusterSnr[i].append(snrFileOne[j])
                    clusterData[i].append(dataFileOne[j].T)
                    spectralDistance[i].append([resultOne[j, 0], resultOne[j, 1]])
            centerS_SC.append(np.array(spectralDistance[i]).mean(axis=0).tolist())
            # print(np.array(kmeansDistance[0]).mean(axis = 0)),列均值，求聚类散点的均值（均值即聚类中心，欧式距离）
        # 存储每类均谱
        centerS_SC = np.array(centerS_SC)
        for i in range(clusters_N):
            plt.scatter(centerS_SC[i][0], centerS_SC[i][1], c="red", s=pcaSize + 6)

        # plt.legend(loc=0, ncol=1)
        plt.xlabel(u"tSNE 1")
        plt.ylabel(u"tSNE 2")
        plt.title('SpectralClustering-tSNE')
        # plt.show()
        outPathSC1 = outPath + '/' + r'_SClustering-tSNE.tif'
        plt.savefig(outPathSC1, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathSC1 = outPath + '/' + r'_SClustering-tSNE.pdf'
        plt.savefig(outPathSC1, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ***************************************************************************************************************计算聚类中心到聚类数据距离

        clusterMeanS = []
        clusterStdS = []
        std1 = []
        std2 = []
        f = open(outPath + '/' + r'_SClustering-tSNE(Clustering evaluation).txt', "w")
        for i in range(clusters_N):
            clusterMeanS.append(np.mean(clusterData[i], axis=0))
            clusterStdS.append(np.std(clusterData[i], axis=0))
            std1.append((np.array(clusterMeanS[i]) - np.array(clusterStdS[i])))
            std2.append((np.array(clusterMeanS[i]) + np.array(clusterStdS[i])))

            f.write("SClustering聚类结果中，第" + str(i + 1) + "堆聚类中心到该堆内所有点欧式距离均值：" + str(
                "%.2f" % (np.mean(cdist([centerS_SC[i].tolist()], spectralDistance[i], metric='euclidean')))) + "\n")

            f.write("SClustering聚类结果中，第" + str(i + 1) + "堆聚类中心到其他所有聚类中心欧式距离均值：" + str(
                "%.2f" % ((np.sum(cdist([centerS_SC[i].tolist()], centerS_SC, metric='euclidean'))) / (
                        len(centerS_SC) - 1))) + "\n")
            # (len(centerS)-1):聚类中心点个数-1，当前点到其他聚类中心的均值
            # cdist([centerS[i].tolist()], centerS, metric='euclidean')，点到其他所有点距离
            # np.sum：距离求和，均值需要除以n-1，即：len(centerS)-1

        f.write(
            "\n\n标签顺序：(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·"
            "(13)正Y(14)左三角形(15)右三角形(16)◇(也可按箱线图或均值方差罗列图颜色对应)")
        f.close()
        # print("kmeans3:", np.mean(cdist([centerS_SC[i].tolist()], kmeansDistance[i], metric='euclidean')))
        #
        # for i in range(0,clusters_N):
        #     print("center3:",(np.sum(cdist([centerS_SC[i].tolist()], centerS_SC, metric='euclidean')))/(len(centerS_SC)-1))
        # **************************************************************************************************************保存谱聚类聚类均值
        hcaclusterMeanS = np.column_stack((x, np.array(clusterMeanS).T))
        np.savetxt(outPath + '/' + r'_SClustering-means.txt',
                   hcaclusterMeanS, delimiter='\t', fmt='%.2f')

        # **************************************************************************************************************根据聚类结果，画均值方差图
        figure(figsize=(8, 12))
        for interI in range(clusters_N):
            interIhight = 0.6
            # print("1:",np.array(x).shape)
            # print("2:",np.array(clusterData[interI]).shape)
            # return 0
            plt.plot(x, (clusterMeanS[interI] + interIhight * interI),
                     color=colorSpectrumColour[interI % len(colorSpectrumColour)], lw=1)
            plt.fill_between(x, (std1[interI]) + interIhight * interI, (std2[interI]) + interIhight * interI,
                             color="#B5B5B5")
        plt.title('MeanStd column chart_SC', fontsize=12)
        # plt.show()

        # plt.legend(loc = 'best')
        outPathSC2 = outPath + '/' + r'_SClustering-tSNEmeanStd.tif'
        plt.savefig(outPathSC2, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathSC2 = outPath + '/' + r'_SClustering-tSNEmeanStd.pdf'
        plt.savefig(outPathSC2, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #
        # for i in range(clusters_N):
        #     meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
        #     # 按照行进行均值拼接
        #     stdSpectrum.append(np.std(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))

        # **************************************************************************************************************根据聚类结果，画箱线图
        # figure()
        dfLists = []
        # 存储以文件为单位信噪比字典
        for interI in range(clusters_N):
            dfLists.append(pd.DataFrame({"cluster:" + str(interI + 1): clusterSnr[interI]}))
        dfData = pd.concat(dfLists, axis=1)

        f = dfData.boxplot(sym='+', patch_artist=True, return_type='dict')

        for box, c in zip(f['boxes'], colorSpectrumColour):
            # 箱体边框颜色
            box.set(color=c, linewidth=2)
            # 箱体内部填充颜色
            box.set(facecolor=c)

        plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.8)

        outPathSC3 = outPath + '/' + r'_SClustering-tSNEbox.tif'
        plt.savefig(outPathSC3, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPathSC3 = outPath + '/' + r'_SClustering-tSNEbox.pdf'
        plt.savefig(outPathSC3, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ***************************************************************************************************************保存txt聚类结果

        f = open(outPath + '/' + r'_SClustering-tSNE.txt', "w")
        for i in range(clusters_N):
            f.write(str(i + 1) + "\t")
            for j in range(len(labelOneResultL[i])):
                # f.write(str((labelOneResultL[i])[j]) + ",")
                f.write(str((labelOneResultL[i])[j]))
                if j < len(labelOneResultL[i]) - 1:
                    f.write(",")
            f.write('\n')

        f.close()

        # ***************************************************************************************************************word
        # print("删除标签：", idLabelOneDel)
        # print("标签：", idLabelOne)
        # print("聚类：", clf.labels_)
        # print("11",labelOneResult)
        # print("22", labelOneResultL)
        document = Document()
        # 打开文档
        document.styles['Normal'].font.name = u'宋体'
        document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
        # 设置文档格式
        # t0：标题
        t0 = document.add_heading(u'实验报告', 1)
        t0.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # 直接添加标题，“1”表示1级标题,标题居中
        # t1,t2,t3次级标题
        t1 = document.add_heading(u'1.数据信息', 2)
        # 增加表格1
        rows_num = 3
        cols_num = 2
        table1 = document.add_table(rows=rows_num, cols=cols_num, style='Table Grid')
        table1.style.font.name = u'宋体'
        listTable1 = ['选中光谱ID', '剔除光谱ID', '保留光谱ID']
        # 列标签
        for i in range(rows_num):
            table1.cell(i, 0).text = listTable1[i]
        # 行信息
        table1.cell(0, 1).text = str(idLabel)
        # table1.cell(1, 1).text = str(snrDelfileOne)
        table1.cell(1, 1).text = str(idLabelOneDel)
        table1.cell(2, 1).text = str(idLabelOne)

        t2 = document.add_heading(u'2.参数信息', 2)
        # 增加表格2
        listTable2 = ["参数名", "参数意义", "参数值"]
        listTable2Name = ["dataPath", "outPath", "startSpec", "endSpec", "snrNoiseindex", "delSNRmin", "delSNRmax",
                          "filterSize",
                          "dynamicFactor", "winSG", "nSG", "lambdaAirPls", "itermaxAirPls", "pltInter",
                          "processLinewidth",
                          "distanceMethod", "hcaLabelsize", "pcaNcomponents", "pca_col1", "pca_col2", "pcaSize",
                          "pcaAlpha",
                          "nDim", "nPerplex", "learnRate", "nInter", "nGrad", "svmTestSize", "svmC", "xKernel",
                          "clusters_N"]
        listTable2Meaning = ["输入路径", "输出路径", "截谱长度(首)", "截谱长度(尾)", "静默区：尾端长度", "信噪比最小阈值", "信噪比最大阈值",
                             "宇宙射线：窗口宽度", "宇宙射线：动态因子", "滤波：窗口宽度", "滤波：拟合阶次",
                             "基线校正：lambda", "基线校正：迭代次数", "显示光谱量(未使用)", "光谱线宽(未使用)",
                             "HCA距离度量方法:\r0:沃德方差最小化\r1:质心欧式距离\r2:加权分组平均\r3:UPGMA算法",
                             "HCA(tSNE)标号大小", "PCA主成分个数", "PCA(tSNE)选择维度", "PCA(tSNE)选择维度",
                             "PCA(tSNE)标签大小", "PCA(tSNE)透明度", "tSNE空间维度", "Perplex", "tSNE学习率",
                             "tSNE最大迭代次数", "tSNE收敛误差", "测试数据占比", "SVM错误项惩罚系数",
                             "SVM核函数:\r0:多项式核\r1:线性核\r2:径向基核\r3:sigmod核\r4:核矩阵", "聚类堆数"]
        listTable2Value = [dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize,
                           dynamicFactor, winSG,
                           nSG, lambdaAirPls, itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize,
                           pcaNcomponents, pca_col1, pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter,
                           nGrad, svmTestSize, svmC, xKernel, clusters_N]

        rows_num = len(listTable2Meaning) + 1
        cols_num = len(listTable2)
        table2 = document.add_table(rows=rows_num, cols=cols_num, style='Table Grid')
        table2.style.font.name = u'宋体'
        # 列标题
        for j in range(0, cols_num):
            table2.cell(0, j).text = listTable2[j]
        # 行标题
        for i in range(1, rows_num):
            table2.cell(i, 0).text = listTable2Name[i - 1]
            table2.cell(i, 1).text = listTable2Meaning[i - 1]
            table2.cell(i, 2).text = str(listTable2Value[i - 1])
        # 均值方差图
        t3 = document.add_heading(u'3.全部数据均值方差图', 2)
        table31 = document.add_table(rows=1, cols=1, style='Table Grid')
        table31.style.font.name = u'宋体'
        table31.cell(0, 0).text = "注：全部光谱数据的均值和标准差"
        # table12.cell(0, 1).text = str("%.2f%%" % (scoreLDA * 100))
        img_name = outPath + '/' + files[0] + r'_meanStdOne.tif'
        fig2 = document.add_picture(img_name, width=Inches(5.0))
        # 向文档里添加图片
        # fig2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # #居中显示
        t7 = document.add_heading(u'4.HCA树状图', 2)
        table71 = document.add_table(rows=1, cols=1, style='Table Grid')
        table71.style.font.name = u'宋体'
        table71.cell(0, 0).text = "注：层次聚类树状图叶子节点（从左到右排列）,详见_hcaclusterLeaves.txt文件。"
        img_name = outPath + '/' + r'_HCAcluster.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # tSNE
        t8 = document.add_heading(u'5.tSNE空间分布图', 2)
        table81 = document.add_table(rows=1, cols=1, style='Table Grid')
        table81.style.font.name = u'宋体'
        # table81.cell(0, 0).text = "注：散点标签见文件夹下_HCA-tSNE.txt文件；\n\t标签顺序(也可按颜色对应)：(1)□(2)◇(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·(13)正Y(14)左三角形(15)右三角形(16)△"
        table81.cell(0,
                     0).text = "注：散点标签见文件夹下_HCA-tSNE.txt文件；\n\t标签顺序(也可按颜色对应)：" \
                               "(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·" \
                               "(13)正Y(14)左三角形(15)右三角形(16)◇"

        img_name = outPath + '/' + r'_HCA-tSNE.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # 层次聚类均值方差图
        t9 = document.add_heading(u'6.层次聚类均值方差罗列图', 2)
        table91 = document.add_table(rows=1, cols=1, style='Table Grid')
        table91.style.font.name = u'宋体'
        table91.cell(0, 0).text = "注：光谱数据从下往上排列，按颜色对应[_HCA-tSNE.txt]光谱聚类点分布"
        img_name = outPath + '/' + r'_HCA-tSNEmeanStd.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # 层次聚类箱线图
        t10 = document.add_heading(u'7.层次聚类信噪比箱线图', 2)
        table101 = document.add_table(rows=1, cols=1, style='Table Grid')
        table101.style.font.name = u'宋体'
        table101.cell(0, 0).text = "注：从左往右按颜色对应方差罗列排列"
        img_name = outPath + '/' + r'_HCA-tSNEbox.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # kmeans均值
        t11 = document.add_heading(u'8.kMeans聚类图', 2)
        table111 = document.add_table(rows=1, cols=1, style='Table Grid')
        table111.style.font.name = u'宋体'
        table111.cell(0,
                      0).text = "注：散点标签见文件夹下[_kMeans-tSNE.txt]文件；\n\t标签顺序(也可按颜色对应)：" \
                                "(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形(12)·" \
                                "(13)正Y(14)左三角形(15)右三角形(16)◇\n\t红色圆点为聚类中心点(平面欧式距离中心)"
        img_name = outPath + '/' + r'_kMeans-TSNE.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # kemans聚类均值方差罗列图
        t92 = document.add_heading(u'9.kMeans聚类均值方差罗列图', 2)
        table912 = document.add_table(rows=1, cols=1, style='Table Grid')
        table912.style.font.name = u'宋体'
        table912.cell(0, 0).text = "注：光谱数据从下往上排列，按颜色对应[_kMeans-tSNE.txt]光谱聚类点分布"
        img_name = outPath + '/' + r'_kMeans-tSNEmeanStd.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # kmeans聚类箱线图
        t102 = document.add_heading(u'10.kMeans聚类信噪比箱线图', 2)
        table102 = document.add_table(rows=1, cols=1, style='Table Grid')
        table102.style.font.name = u'宋体'
        table102.cell(0, 0).text = "注：从左往右按颜色对应方差罗列排列"
        img_name = outPath + '/' + r'_kMeans-tSNEbox.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # tSNE-spectralCluster
        tSC = document.add_heading(u'11.SpectralCluster-tSNE空间分布图', 2)
        tableSC1 = document.add_table(rows=1, cols=1, style='Table Grid')
        tableSC1.style.font.name = u'宋体'
        tableSC1.cell(0, 0).text = "注：散点标签见文件夹下_SClustering-tSNE.txt文件；\n\t标签顺序(也可按颜色对应)：" \
                                   "(1)□(2)△(3)○(4)＋(5)▽(6)长菱形(7)×(8)☆(9)正六边形(10)左Y(11)正五边形" \
                                   "(12)·(13)正Y(14)左三角形(15)右三角形(16)◇"
        img_name = outPath + '/' + r'_SClustering-tSNE.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # 谱聚类均值方差图
        tSCstd = document.add_heading(u'12.谱聚类均值方差罗列图', 2)
        tableSCstd1 = document.add_table(rows=1, cols=1, style='Table Grid')
        tableSCstd1.style.font.name = u'宋体'
        tableSCstd1.cell(0, 0).text = "注：光谱数据从下往上排列，按颜色对应[_SClustering-tSNE.txt]光谱聚类点分布"
        img_name = outPath + '/' + r'_SClustering-tSNEmeanStd.tif'
        document.add_picture(img_name, width=Inches(5.0))

        # 谱聚类箱线图
        t10 = document.add_heading(u'13.谱聚类信噪比箱线图', 2)
        table101 = document.add_table(rows=1, cols=1, style='Table Grid')
        table101.style.font.name = u'宋体'
        table101.cell(0, 0).text = "注：从左往右按颜色对应方差罗列排列"
        img_name = outPath + '/' + r'_SClustering-tSNEbox.tif'
        document.add_picture(img_name, width=Inches(5.0))

        document.save(outPath + '/' + r"HookeReport_Onefile.docx")
        # 保存文档
        return 1
    # *******************************************************************************************************************************多个文件夹
    else:
        # 1读取文夹并合并所有文件
        dirList = []
        # 作为变量名
        filesDataName = copy1.copy(files)
        # 作为同名变量数据
        pcaCountAll = []
        # 用于存放每个标签数量
        pcaCount = []
        # 用于存储删除光谱后标签数量
        # i = 0
        filesNumInter = 0
        # filesNumInter 全局文档个数，迭代时“从0开始，最后加1”
        classLabel = []
        # 创建标签
        classLabelName = []
        # 和标签对应的名字
        classOneLabel = []
        # 创建单个标签
        snrAllfile = []
        snrAllfile1 = []
        # 存储所有文件光谱信噪比
        allData = []
        # 用于存储所有处理后的数据
        # print(len(classOneLabel))
        meanAllSpectrum = []
        # 用于存储所有文件光谱均值，画均值方差层次罗列图
        stdAllSpectrum = []
        # 用于保存标准差到txt
        stdAll1 = []
        stdAll2 = []
        # 分别用于存储均值方差图上下bar，阴影部分
        snrDelfile = []
        # 用于存储删除信噪比小于阈值的光谱
        for f in files:
            filesDataName[filesNumInter] = []
            pltY = []
            # 存储光谱y值，用于有层次画图
            meanSpectrum = []
            # 存储该文件下光谱均值
            stdSpectrum = []
            # 存储该文件夹下光谱标准差
            if os.path.isdir(dataPath + '/' + f):  # 判断是不是文件夹
                # if f[0] != '.':  # 排除隐藏文件夹。因为隐藏文件夹过多
                dirList.append(f)  # 添加非隐藏文件夹

            filesDataName[filesNumInter] = readMerge(dataPath[:] + "/" + dirList[filesNumInter])
            # print("***",np.array(filesDataName).shape)
            # return 0
            filesDataNameTemp = filesDataName[filesNumInter]
            # allData = readMerge(dataPath[:] + "\\" + dirList[i])
            pcaCountAll.append(filesDataNameTemp.shape[1] - 1)
            # print('pcaCount:',pcaCount)
            # 把该标签数据量添加到pcaCount中

            filesDataNameTemp = filesDataNameTemp[startSpec: (filesDataNameTemp.shape[0] - endSpec), :]
            # #截谱
            # # **************************************************************************************************前后截谱
            # if filesNumInter == 0:
            #     filesDataNameTemp = filesDataNameTemp[100:, :]
            # elif filesNumInter == 1:
            #     filesDataNameTemp = filesDataNameTemp
            # elif filesNumInter == 2:
            #     filesDataNameTemp = filesDataNameTemp[100:, :]
            # else:
            #     filesDataNameTemp = filesDataNameTemp

            filesDataName1 = copy1.copy(filesDataNameTemp)
            # 注意：直接采用 ’ = ’ 赋值方式会使得两个变量占用同一地址
            x = filesDataNameTemp[:, 0]

            snrOnefile = []
            # 存储单个文件下拉曼光谱信噪比
            snrOneDelfile = []
            # 用于存储单个文件下删除信噪比小于阈值的光谱

            countDelSpect = 0
            # 用于存储该文件夹下被删除文件的数量
            # 2批处理
            for j in range(1, filesDataNameTemp.shape[1]):
                snrS = 0
                # 初始化该光谱信噪比
                # 循环处理每个光谱
                y = filesDataNameTemp[:, j]
                # 从第二列开始，保留了第一列坐标，后面需要去掉
                # y1 = Norris(y, nNorris)

                y1 = CRR(y, filterSize, dynamicFactor)
                y2 = savgol_filter(y1, winSG, nSG)
                # 2.3 airPLS
                # print("j:",j)
                y3 = airPLS(y2, lambdaAirPls, itermaxAirPls)
                # 有参去背景（自适应迭代重加权惩罚最小二乘，airpls）
                # y3 = background_substruction(y2)
                # 无参去背景（交替最小二乘ALS）
                # 2.4 归一化
                y4 = MaxMinNormalization(y3)
                # snrNoiseindex = 20
                # # 从尾部选取长度，作为静默区
                # delSNR = 10
                # #删除光谱信噪比阈值

                # snrSpectrumSingal = max(y4[170 - startSpec:400 - startSpec]) - min(y4)
                snrSpectrumSingal = max(y4) - min(y4)
                # 信号强度
                snrSpectrumNoise = max(y4[-snrNoiseindex:]) - min(y4[-snrNoiseindex:])
                # 噪声强度
                snrS = snrSpectrumSingal / snrSpectrumNoise
                # 光谱信噪比
                if snrS < delSNRmin or snrS > delSNRmax:
                    snrOneDelfile.append(j)
                    countDelSpect = countDelSpect + 1
                    continue

                snrOnefile.append(snrS)
                snrAllfile1.append(snrS)
                # 全部信噪比值存储到一个list中

                filesDataNameTemp[:, j - countDelSpect] = y4
                # j - countDelSpect,用于减去删除光谱的数量
                # if j <= pltInter:
                #     pltTemp = y4 + j * 0.3
                #     pltY.append(pltTemp)
                #     # 用于层次化画光谱
                #     # np.column_stack((Data_x, Dout.T))
                classLabel.append(filesNumInter)
                classLabelName.append(files[filesNumInter])

                # print("snrOnefile",snrOnefile)
            filesDataNameTemp = filesDataNameTemp[:, 0:(filesDataNameTemp.shape[1] - countDelSpect)]

            # ************************************************************************************************************保存处理后数据
            np.savetxt(outPath + '/' + r'Process_' + files[filesNumInter] + '.txt',
                       filesDataNameTemp, delimiter='\t', fmt='%.2f')

            # print("删除文件数量：",countDelSpect)
            snrDelfile.append(snrOneDelfile)
            # 用于存储所有信噪比小于阈值的光谱
            pcaCount.append(filesDataNameTemp.shape[1] - 1)
            # allData.append(filesDataNameTemp)
            # 存储所有处理后的光谱数据
            snrAllfile.append(snrOnefile)
            # 按文件存储所有光谱信噪比
            # print("filesDataNameTemp:",np.array(filesDataNameTemp).shape)
            # print("pltY",np.array(pltY).shape)
            classOneLabel.append(filesNumInter)
            # 存储标签名，只包含1个
            # urldic = urlparse(dataPath)
            # pathdict = urldic.path.split('\\')
            # # 以"\"分割路径，供后文输出路径地址
            # outPath1 = outPath + '\\' + files[i] + r'_process.txt'
            # 输出地址及其文件名
            filesDataName[filesNumInter] = filesDataNameTemp
            # np.savetxt(outPath1, filesDataName[i], fmt='%f', delimiter=' ')

            # **********************************************************************************************************批处理画图,处理前
            # figure(3 * filesNumInter)
            # # processLinewidth = 0.5
            # # #线宽
            # endPlt = min(pltInter, filesDataName1.shape[1])
            # plt.plot(x, filesDataName1[:, 1:endPlt], linewidth=processLinewidth)
            # # 显示网格
            # plt.grid(linestyle='-.')
            # plt.tight_layout()
            # # plt.show()
            # outPath1 = outPath + '\\' + files[filesNumInter] + r'_origin.tif'
            # plt.savefig(outPath1, bbox_inches='tight')
            # **********************************************************************************************************批处理画图处理后
            # figure(3 * filesNumInter + 1)
            # plt.plot(x, np.array(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]]), linewidth=processLinewidth)
            # # 显示网格
            # plt.grid(linestyle='-.')
            # plt.tight_layout()
            # # plt.show()
            # outPath2 = outPath + '\\' + files[filesNumInter] + r'_process.tif'
            # plt.savefig(outPath2, bbox_inches='tight')
            # # print("111:",np.array(filesDataNameTemp).shape)
            # **********************************************************************************************************处理后均值方差图
            figure(3 * filesNumInter + 2)
            meanSpectrum.append(np.mean(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))
            # 按照行进行均值拼接
            stdSpectrum.append(np.std(filesDataNameTemp[:, 1:filesDataNameTemp.shape[1]], axis=1))

            # # 查找标准差最大处的峰位
            # print(np.array(stdSpectrum).shape)
            # print("1:",stdSpectrum[0].tolist())
            # # print(x[stdSpectrum.index(max(stdSpectrum[0], key=abs))])
            # print(x[stdSpectrum[0].tolist().index(max(stdSpectrum[0].tolist(), key=abs))])

            # 按照行进行标准差拼接
            stdAllSpectrum.append(stdSpectrum)
            # 拼接标准差
            plt.plot(x, np.array(meanSpectrum).T, color=colorSpectrumColour[filesNumInter % len(colorSpectrumColour)],
                     lw=1,
                     label='Spectral mean')
            # plt.plot(x, np.array(meanSpectrum).T, color="white", lw=2)
            # print(np.array(x).shape)
            # print("111:",(np.array(meanSpectrum).T).shape)
            # print("222:",((np.array(meanSpectrum) - np.array(stdSpectrum)).T).shape)
            std1 = (np.array(meanSpectrum) - np.array(stdSpectrum)).T
            std2 = (np.array(meanSpectrum) + np.array(stdSpectrum)).T
            plt.fill_between(x, std1.flatten(), std2.flatten(), color="#B5B5B5", label='Std of SNR')
            plt.legend(loc='best')
            # flatten()二维转一维
            outPath3 = outPath + '/' + files[filesNumInter] + r'_meanStd.tif'
            plt.savefig(outPath3, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath3 = outPath + '/' + files[filesNumInter] + r'_meanStd.pdf'
            plt.savefig(outPath3, format='pdf', bbox_inches='tight', transparent=True, dpi=300)

            meanAllSpectrum.append(np.array(meanSpectrum).T)
            # # 用于存储所有文件光谱均值，画均值方差层次罗列图
            stdAll1.append(std1.flatten())
            # a是个数组，a.flatten()就是把a降到一维，默认是按行的方向降
            stdAll2.append(std2.flatten())
            # **********************************************************************************************************保存均值
            # print("???",type(x))
            # print("???",type(y))
            xMean = np.array(x).reshape(-1, 1)
            yMeam = np.array(meanSpectrum).reshape(-1, 1)
            # print(xMean.shape)
            # print(yMeam.shape)
            dataMean = np.hstack((xMean, yMeam))

            np.savetxt(outPath + '/' + r'Mean_' + files[filesNumInter] + '.txt',
                       dataMean, delimiter='\t', fmt='%.2f')

            # plt.show()

            filesNumInter = filesNumInter + 1
            plt.close('all')
        # print("allData1:",np.array(allData).shape)
        # **********************************************************************************************将信噪比保存到CSV,信噪比箱型抖动图

        snrCSV = np.array(snrAllfile1).reshape(-1, 1)
        labelCSV = np.array(classLabelName).reshape(-1, 1)
        SNRClass = np.column_stack((snrCSV, labelCSV))
        outPuthExcSNR = outPath + '/' + r'SNR.csv'
        snrList = pd.DataFrame(SNRClass, columns=["SNR", "LabelName"])
        snrList.to_csv(outPuthExcSNR)

        dfBox = pd.read_csv(outPuthExcSNR)
        sns.boxplot(x='LabelName', y='SNR', data=dfBox)
        # sns.violinplot(x='SNR', y='LabelName', data=df)
        # 小提琴形状替换箱线图
        sns.stripplot(x='LabelName', y='SNR', data=dfBox, jitter=0.25, palette='Set2', dodge=True, size=4, alpha=0.5,
                      color=None, )
        # 可以再加一个分量hue，可用来控制每组抖动点进一步划分不同颜色！！！！！
        # linewidth=1,加水平网格
        # plt.legend()
        plt.xticks(fontsize=12, rotation=90)
        plt.yticks(fontsize=12)
        plt.xlabel('Labels', fontsize=12)  # 添加x轴的名称
        plt.ylabel('SNR intensity', fontsize=12)
        # plt.show()
        # plt.ylim([1.5,15])
        # # 设置坐标范围

        outPath6 = outPath + '/' + r'_snrBox1.tif'
        plt.savefig(outPath6, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath6 = outPath + '/' + r'_snrBox1.pdf'
        plt.savefig(outPath6, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # **************************************************************************************************将均谱保存txt
        meanAllSpectrumTuple0 = np.array(meanAllSpectrum).shape[0]
        # tuple索引
        meanAllSpectrumTuple1 = np.array(meanAllSpectrum).shape[1]
        meanAllSpectrumTxt = np.array(meanAllSpectrum).reshape(meanAllSpectrumTuple0, meanAllSpectrumTuple1).T
        # reshape变形，(m,n,1)到(m,n)
        meanAllSpectrumTxt = np.column_stack((x.T, meanAllSpectrumTxt))
        # 列拼接
        np.savetxt(outPath + '/' + r'_spectrumMean.txt', meanAllSpectrumTxt, delimiter='\t', fmt='%.2f')

        # **************************************************************************************************************均值方差层次罗列图
        figure(3 * filesNumInter, figsize=(8, 12))

        # data = np.array(
        #     pd.read_csv(r"C:\Users\lixin\Desktop\11.txt", delim_whitespace=True,
        #                 header=None))
        # x = data[50:,0]

        for interI in range(filesNumInter):
            interIhight = 0.6
            plt.plot(x, meanAllSpectrum[interI] + interIhight * interI,
                     color=colorSpectrumColour[interI % len(colorSpectrumColour)], lw=1, label=files[interI])
            plt.fill_between(x, stdAll1[interI] + interIhight * interI, stdAll2[interI] + interIhight * interI,
                             color="#B5B5B5")
            # plt.title('MeanStd column chart', fontsize=12)
            plt.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
                       mode="expand", borderaxespad=0, ncol=3)
        plt.yticks([])
        # 不显示纵坐标
        # plt.legend(loc = 2)
        plt.rcParams.update({'font.size': 12})
        # plt.show()
        outPath4 = outPath + '/' + r'_meanStdcolumn.tif'
        plt.savefig(outPath4, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath4 = outPath + '/' + r'_meanStdcolumn.pdf'
        plt.savefig(outPath4, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ****************************************************************************************************************保存标准差信息
        # stdAllSpectrum
        f = open(outPath + '/' + r'_meanStdcolumn.txt', "w")
        f.write('保存同类拉曼光谱所有拉曼位移的标准差' + '\n')
        for interI in range(filesNumInter):
            # f.write('保存同类拉曼光谱所有拉曼位移的标准差：' + '\n')
            f.write("[" + str(files[interI]) + ']光谱的平均标准差为：' + str(round(np.mean(stdAllSpectrum[interI]), 2)) + '\n')
            # f.write("[" + str(files[interI]) + ']光谱的平均标准差为：' + str(round(np.mean(stdAll1[interI]), 2)) + '\n')

            f.write("[" + str(files[interI]) + ']光谱的最大标准差为：' + str(
                round(np.max(stdAllSpectrum[interI]), 2)) + ",最大处峰位：" + str(
                x[stdAllSpectrum[interI][0].tolist().index(max(stdAllSpectrum[interI][0].tolist(), key=abs))]) + '\n')
            # x[stdSpectrum[0].tolist().index(max(stdSpectrum[0].tolist(), key=abs))]

            # f.write("[" + str(files[interI]) + ']光谱的最大标准差为：' + str(
            #     round(np.max(stdAll1[interI]), 2)) + ",最大处峰位：" + str(
            #     x[stdAll1[interI].index(max(stdAll1[interI][interI], key=abs))]) + '\n')
        f.close()
        # **************************************************************************************************************信噪比直方图
        figure(3 * filesNumInter + 1)
        snrAlpha = 0.5
        snrLinewidth = 0.8

        for interI in range(filesNumInter):
            sns.distplot(snrAllfile[interI], color=colorSpectrumColour[interI % len(colorSpectrumColour)],
                         label=files[interI],
                         # hist_kws={'alpha': snrAlpha},
                         hist=False,
                         # 不显示直方图
                         # kde=False,
                         # 不显示密度图
                         kde_kws={'linewidth': snrLinewidth})
        plt.title('SNR Density Curves with Histogram', fontsize=12)
        plt.legend(fontsize=8)
        plt.xlabel("SNR")
        plt.ylabel("Density")
        outPath5 = outPath + '/' + r'_snrHistogram.tif'
        plt.savefig(outPath5, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath5 = outPath + '/' + r'_snrHistogram.pdf'
        plt.savefig(outPath5, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # **************************************************************************************************************信噪比箱型图
        figure(3 * filesNumInter + 2)

        dfLists = []
        # 存储以文件为单位信噪比字典
        for interI in range(filesNumInter):
            dfLists.append(pd.DataFrame({files[interI]: snrAllfile[interI]}))
        dfData = pd.concat(dfLists, axis=1)

        # color = dict(boxes='DarkGreen', whiskers='DarkOrange', medians='DarkBlue', caps='Gray')
        #
        # #字典拼接
        # dfData.plot.box(color = color)
        # dfData.boxplot(sym='+', patch_artist=True,
        #                boxprops={"color": colorSpectrumColour[interI % 13], "facecolor": "#F43D68"},
        #                meanprops={"marker": "D", "markerfacecolor": "white"}, medianprops={"linestyle": "--", "color": "r"})
        # plt.grid(linestyle="--", alpha=0.8)

        f = dfData.boxplot(sym='+', patch_artist=True, return_type='dict')

        # 这里共有四个box
        # color = ['k', 'g', 'r', 'deepskyblue']  # 有多少box就对应设置多少颜色

        for box, c in zip(f['boxes'], colorSpectrumColour):
            # 箱体边框颜色
            box.set(color=c, linewidth=2)
            # 箱体内部填充颜色
            box.set(facecolor=c)

        # plt.show()
        # for interI in range(i):
        #     sns.boxplot(x=classOneLabel[interI],y=snrAllfile[interI], notch=False)
        # print(len(classLabel))

        # print("111:",np.array(snrAllfile).shape)
        # df = pd.DataFrame(np.array(snrAllfile).T,
        #                   columns=files)

        # print("111:",np.array(snrAllfile).shape)
        # #数组重塑大小
        # Data = pd.DataFrame(np.array(snrAllfile).reshape(1,3),columns = files)
        # print(Data)
        # Data.boxplot()
        # plt.boxplot(snrAllfile, classLabel)
        # 按照标签命名
        # plt.boxplot(snrAllfile, classLabelName)
        # print(classLabelName)
        # notch=None,    # 是否是凹口的形式展现箱线图，默认非凹口；
        # sym=None,    # 指定异常点的形状，默认为+号显示；
        plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.8)
        outPath6 = outPath + '/' + r'_snrBox.tif'
        plt.savefig(outPath6, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath6 = outPath + '/' + r'_snrBox.pdf'
        plt.savefig(outPath6, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # **********************************************************************************************************拼接
        allData = filesDataName[0]
        for interI in range(1, filesNumInter):
            # tempData = filesDataName[interI]
            allData = np.column_stack((allData, (filesDataName[interI])[:, 1:]))
        ySpec = allData[:, 1:allData.shape[1]].T

        # **************************************************************************************************************3 hca层次聚类
        figure(3 * filesNumInter + 3)
        if distanceMethod == 0:
            distM = "ward"
            # 沃德方差最小化算法
        elif distanceMethod == 1:
            distM = "centroid"
            # 质心间的欧式距离
        elif distanceMethod == 2:
            distM = "weighted"
            # 加权分组平均法
        else:
            distM = 'average'
            # UPGMA算法（非加权组平均）法

        # distanceMethod = 'ward'
        # # 层次聚类距离度量选用的方法,可选：'centroid','weighted','average'
        Z = hierarchy.linkage(ySpec, method=distM, metric='euclidean')
        # 执行层次/聚集聚类
        hcaColorThreshold = 40
        # color_threshold= 8 设置颜色高度阈值，用于控制决策树和PCA的颜色分类
        # hcaLabelsize = 6
        # #标号大小
        # cluster.hierarchy.cut_tree(Z, n_clusters=filesNumInter)
        # 设置聚类个数n_clusters

        hierarchy.dendrogram(Z, color_threshold=hcaColorThreshold, labels=pd.DataFrame(ySpec).index + 1)
        # 将分层聚类绘制为树状图
        # hierarchy.dendrogram(Z, labels=pd.DataFrame(ySpec).index + 1)
        # label = cluster.hierarchy.cut_tree(Z, height = hcaColorThreshold)
        hacLeaves = hierarchy.leaves_list(Z) + 1
        # 返回叶子节点

        plt.xticks(rotation=90)
        plt.tick_params(labelsize=hcaLabelsize)
        # 调整刻度尺寸
        plt.title('HCA')
        # plt.show()
        outPath7 = outPath + '/' + r'_HCA.tif'
        plt.savefig(outPath7, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath7 = outPath + '/' + r'_HCA.pdf'
        plt.savefig(outPath7, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        # plt.show()
        plt.close()
        # *******************************************************************************************保存层次聚类叶子节点
        f = open(outPath + '/' + r'_hcaLeaves.txt', "w")
        f.write("层次聚类树状图叶子节点（从左到右排列）：")
        f.write('\n')
        f.write(str(hacLeaves))
        f.close()

        # **************************************************************************************************************4 PCA数据压缩
        figure()
        # 根据两个最大的主成分进行绘图
        pcaNcomponents = 4
        # 主成分个数
        pca = skldec.PCA(n_components=pcaNcomponents)
        # 第一种，选择指定降维方差：n_components =0.95，贡献率之和到95%
        # 第二种，选择指定降维后的维数：n_components =5
        try:
            pca.fit(ySpec)
        except:
            pca.fit(ySpec)
        # pca.fit(ySpec)
        # 主城分析时每一行是一个输入数据
        result = pca.transform(ySpec)

        # #**********************************************************************二维pca标记
        #
        tsneCount1 = 0
        tsneCount2 = 0

        # pcaSize = 30
        # #pca标签大小
        # pcaAlpha = 1
        # #pca透明度
        for tsneI in range(len(classOneLabel)):
            # print("光谱名称：",files[i])
            # print(type(files[i]))
            # print("光谱数量：", filesDataName[i].shape)
            tsneCount2 = tsneCount2 + pcaCount[tsneI]
            #plt.scatter(result[tsneCount1:tsneCount2, pca_col1], result[tsneCount1:tsneCount2, pca_col2],
            #            c=colorSpectrumColour[tsneI % len(colorSpectrumColour)], marker=markers[tsneI % len(markers)],
            #            s=pcaSize,
            #            alpha=pcaAlpha, edgecolor='k', label=files[tsneI])
            plt.scatter(result[tsneCount1:tsneCount2, pca_col1], result[tsneCount1:tsneCount2, pca_col2],
                        c=colorSpectrumColour[tsneI % len(colorSpectrumColour)], marker=markers[tsneI % len(markers)],
                        s=pcaSize,
                        alpha=pcaAlpha, label=files[tsneI])
            tsneCount1 = tsneCount1 + pcaCount[tsneI]

        # # 二维标记
        # for i in range(0,len(hacLeaves)):
        #     plt.text(result[i, 0], result[i, 1], i + 1, fontsize=10)
        # plt.show()

        plt.legend(loc=0, ncol=1, fontsize=8)
        # plt.legend(loc=0, ncol=1, fontsize=5)
        # set(tsneL, 'FontName', 'Times New Roman', 'FontSize', 5, 'FontWeight', 'normal')
        plt.xlabel(u"PC 1")
        plt.ylabel(u"PC 2")
        plt.title('PCA_2D')
        # plt.show()
        outPath71 = outPath + '/' + r'_PCA_2D.tif'
        plt.savefig(outPath71, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath71 = outPath + '/' + r'_PCA_2D.pdf'
        plt.savefig(outPath71, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # # **************************************************************************************************二元关系分析
        #
        # 修改类别形状，由一维变二维（n,）变为(n,1),方便后续列拼接
        classLabelName2 = np.array(classLabelName).reshape(-1, 1)
        tsneClass = np.column_stack((result, classLabelName2))
        # np.savetxt(r"C:\Users\lixin\Desktop\b\tsne.txt",
        #            result2, delimiter='\t', fmt='%.2f')
        outPuthExcPCA = outPath + '/' + r'PCA.csv'
        # allDataTwoClass = np.column_stack(
        #     (np.array(xLabelTest), np.array(yLabelTest), np.array(testTarget), np.array(knn.predict(testData)),
        #      np.array(knn.predict(testData)), np.array(clf.predict(testData))))
        # 存储前4维PCA
        tsneList = pd.DataFrame(tsneClass, columns=['PC1', 'PC2', 'PC3', 'PC4', 'labels'])
        tsneList.to_csv(outPuthExcPCA, index=False)
        # 调用保存的tsne表格,index=False:不要序号
        # df = pd.read_csv(outPuthExc)
        # dfTSNE = pd.read_csv(outPuthExcTSNE).iloc[:,1:5]
        # 切片
        dfPCA = pd.read_csv(outPuthExcPCA)

        g = sns.pairplot(dfPCA, kind="scatter", diag_kind="kde", diag_kws=dict(bw=1.5), hue="labels", palette="hls",
                         plot_kws=dict(s=12, edgecolor="black", linewidth=0.3))
        # g = sns.pairplot(dfPCA, kind="scatter", diag_kind="kde", hue="labels", palette="hls",
        #                  plot_kws=dict(s=15, edgecolor="black", linewidth=0.5),kde_kws = {'bw' : 1.5})
        # g.map_diag(sns.kdeplot)
        # kind：用于控制非对角线上的图的类型，可选"scatter"与"reg"
        # diag_kind：控制对角线上的图的类型，可选"hist"(直方图)与"kde"（密度图），加上diag_kws=dict(bw=1.5),避免报错
        # hue ：针对某一字段进行分类
        # palette：控制色调
        g.map_offdiag(sns.kdeplot, n_levels=2)
        # sns.kdeplot(x,y,cmap=pal)：绘制核密度分布图
        # plt.show()
        outPath91 = outPath + '/' + r'_pcaPair.tif'
        plt.savefig(outPath91, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath91 = outPath + '/' + r'_pcaPair.pdf'
        plt.savefig(outPath91, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ********************************************************************************************************PCA三维
        figure(3 * filesNumInter + 4)
        ax = plt.figure().add_subplot(111, projection='3d')
        # 其中参数111，指的是将图像分成1行1列，此子图占据从左到右从上到下的1位置
        pcaNcomponents3D = nDim + 1
        # 主成分个数
        pca_3D = skldec.PCA(n_components=pcaNcomponents3D)
        # 第一种，选择指定降维方差：n_components =0.95，贡献率之和到95%
        # 第二种，选择指定降维后的维数：n_components =5
        # pca_3D.fit(ySpec)
        try:
            pca_3D.fit(ySpec)
        except:
            pca_3D.fit(ySpec)
        # 主城分析时每一行是一个输入数据
        result_3D = pca_3D.transform(ySpec)

        pca_col3 = 2
        pcaCount1 = 0
        pcaCount2 = 0
        # pcaSize = 30
        # #pca标签大小
        # pcaAlpha = 1
        # #pca透明度
        for pcai in range(len(classOneLabel)):
            pcaCount2 = pcaCount2 + pcaCount[pcai]
            ax.scatter(result_3D[pcaCount1:pcaCount2, pca_col1], result_3D[pcaCount1:pcaCount2, pca_col2],
                       result_3D[pcaCount1:pcaCount2, pca_col3],
                       c=colorSpectrumColour[pcai % len(colorSpectrumColour)], marker=markers[pcai % len(markers)],
                       s=pcaSize,
                       alpha=pcaAlpha, edgecolor='k', label=files[pcai])
            pcaCount1 = pcaCount1 + pcaCount[pcai]
        # 设置坐标轴
        plt.legend(loc='best', fontsize=8)
        ax.set_xlabel('PC1')
        ax.set_ylabel('PC2')
        ax.set_zlabel('PC3')
        plt.title('PCA_3D')
        # plt.show()
        # return 0
        outPath16 = outPath + '/' + r'_PCA_3D.tif'
        plt.savefig(outPath16, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath16 = outPath + '/' + r'_PCA_3D.pdf'
        plt.savefig(outPath16, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        ##**************************************************************************************************************tsne降维
        figure(3 * filesNumInter + 5)
        # 根据两个最大的主成分进行绘图
        pcaNcomponents = 4
        # 主成分个数
        # pca = skldec.PCA(n_components=pcaNcomponents)
        # # 第一种，选择指定降维方差：n_components =0.95，贡献率之和到95%
        # # 第二种，选择指定降维后的维数：n_components =5
        # pca.fit(ySpec)
        # # 主城分析时每一行是一个输入数据
        # result = pca.transform(ySpec)
        nDim = 2
        # int, 空间的维度
        nPerplex = 10.0
        # 数据集越大，需要参数值越大，建议值位5 - 50
        learnRate = 200
        # float,学习率，建议取值为10.0-1000.0
        nInter = 1000
        # int, 最大迭代次数
        nGrad = 1e-06
        # float,如果梯度低于该值，则停止算法

        result2 = TSNE(n_components=nDim, perplexity=nPerplex, early_exaggeration=12.0, learning_rate=learnRate,
                       n_iter=nInter,
                       n_iter_without_progress=300, min_grad_norm=nGrad, metric='euclidean', init='pca',
                       verbose=0, random_state=1, method='exact').fit_transform(ySpec)
        # random_state:int, RandomState or None, default:None，伪随机数生成器种子,不同的初始化可能会导致成本函数的局部最小值不同

        # print(type(result2))

        # pca_col1 = 0
        # pca_col2 = 1
        tsneCount1 = 0
        tsneCount2 = 0

        # pcaSize = 30
        # #pca标签大小
        # pcaAlpha = 1
        # #pca透明度
        for tsneI in range(len(classOneLabel)):
            # print("光谱名称：",files[i])
            # print(type(files[i]))
            # print("光谱数量：", filesDataName[i].shape)
            tsneCount2 = tsneCount2 + pcaCount[tsneI]
            plt.scatter(result2[tsneCount1:tsneCount2, pca_col1], result2[tsneCount1:tsneCount2, pca_col2],
                        c=colorSpectrumColour[tsneI % len(colorSpectrumColour)], marker=markers[tsneI % len(markers)],
                        s=pcaSize,
                        alpha=pcaAlpha, label=files[tsneI])
            tsneCount1 = tsneCount1 + pcaCount[tsneI]

        # for i in range(100):
        #     plt.text(result2[i, 0], result2[i, 1], i + 1, fontsize=14)
        # plt.show()
        plt.legend(loc=0, ncol=1, fontsize=8)
        # plt.legend(loc=0, ncol=1, fontsize=5)
        # set(tsneL, 'FontName', 'Times New Roman', 'FontSize', 5, 'FontWeight', 'normal')
        plt.xlabel(u"tSNE 1")
        plt.ylabel(u"tSNE 2")
        plt.title('t-SNE')
        # plt.show()
        outPath9 = outPath + '/' + r'_tSNE.tif'
        plt.savefig(outPath9, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath9 = outPath + '/' + r'_tSNE.pdf'
        plt.savefig(outPath9, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ****************************************************************************************************************按文件split 分割 & 拼接
        # Target0 = np.zeros(np.array(filesDataName[0][:, 1:]).shape[1])
        # print(type(Target0))
        # print(Target0)
        Target0 = np.full(np.array(filesDataName[0][:, 1:]).shape[1], 0, dtype=int)
        # 创建首个文件label,标签转置，转换成和数据同格式
        # print(Target0)
        # print((filesDataName[0][:, 1:].T).shape)
        # print((Target0.T).shape)
        trainData, testData, trainTarget, testTarget = train_test_split((filesDataName[0][:, 1:].T), (Target0.T),
                                                                        test_size=svmTestSize, random_state=1)
        # train_test_split交叉验证中常用的函数
        # svmC = 0.8
        # # 错误项的惩罚系数,范围[0.5-1],C越大分类效果越好，但有可能会过拟合（defaul C = 1）
        # 首个文件数据分割
        for interI in range(1, filesNumInter):
            #
            # TargetTep = np.zeros(np.array(filesDataName[interI][:, 1:]).shape[1])
            TargetTep = np.full(np.array(filesDataName[interI][:, 1:]).shape[1], interI, dtype=int)
            # print(TargetTep)
            # 创建其他文件label#,标签转置，转换成和数据同格式
            trainDataTep, testDataTep, trainTargetTep, testTargetTep = train_test_split(
                (filesDataName[interI][:, 1:].T), TargetTep.T,
                test_size=svmTestSize, random_state=1)
            # 其他文件数据分割并拼接
            trainData = np.row_stack((trainData, trainDataTep))
            testData = np.row_stack((testData, testDataTep))
            trainTarget = np.concatenate((trainTarget, trainTargetTep), axis=0)
            # 一维数组垂直拼接
            testTarget = np.concatenate((testTarget, testTargetTep), axis=0)
            # 一维数组垂直拼接

        # print(trainData.shape)
        # print(testData.shape)
        # print(trainTarget.shape)
        # print(testTarget.shape)

        # dataTarget1 = label_binarize(trainTarget, np.arange(len(classOneLabel)))
        testTarget1 = label_binarize(testTarget, classes=np.arange(len(classOneLabel)))
        # 标签二值化，为了将ROC曲线和ROC区域扩展到多类或多标签分类，有必要对输出进行二值化：
        # ⑴可以每个标签绘制一条ROC曲线。
        # ⑵也可以通过将标签指示符矩阵的每个元素视为二元预测（微平均）来绘制ROC曲线。
        # ⑶另一种用于多类别分类的评估方法是宏观平均，它对每个标签的分类给予相同的权重。

        # return 0
        # trainData, testData, trainTarget, testTarget = train_test_split(ySpec, dataTarget,
        #                                                                 test_size=svmTestSize, random_state=1)
        # **************************************************************************************************************SVM混淆矩阵
        figure(3 * filesNumInter + 6)
        # 分类SVM、LDA、KNN
        # print("数据大小",ySpec.shape)
        # classData = np.column_stack((ySpec, np.array(classLabel).T))
        # 将标签拼接到数据中
        # print("拼接标签后的数据大小：",classData.shape)
        # *********************************************************************************************网格搜索和交叉验证
        # modelSVM = svm.SVC(C=svmC,probability=True)
        modelSVM = svm.SVC(decision_function_shape='ovr', probability=True, class_weight='balanced')
        # class_weight = {1: 10},class_weight='balanced'
        # kernel='linear'时，为线性核，C越大分类效果越好，但有可能会过拟合（defaul C=1）。
        # kernel='rbf'时（default），为高斯核，gamma值越小，分类界面越连续；
        # gamma值越大，分类界面越“散”，分类效果越好，但有可能会过拟合。
        # kernel='rbf'时（default），为高斯核，；

        # 建立需要搜索的参数的范围
        param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                      'C': [0.7, 0.8, 1]}
        grid_search = GridSearchCV(modelSVM, param_grid, cv=3)
        try:
            clf = grid_search.fit(trainData, trainTarget)
        except:
            clf = grid_search.fit(trainData, trainTarget)
        # clf = grid_search.fit(trainData, trainTarget)
        # 训练模型

        # *******************************************************************************************保存、加载分类器模型
        # 保存模型
        outPathSVM = outPath + '/' + r'_svmModel.pkl'
        joblib.dump(clf, outPathSVM)
        # 加载模型
        # outPathSVM = outPath + '\\' + r'clfSVM.pkl'
        # clfSVM = joblib.load(outPathSVM)
        best_model_svm = grid_search.best_estimator_
        # print("***",type(grid_search.best_params_))
        # print(type(grid_search.best_params_))
        # print("默认超参数：",best_model)
        # print(type(best_model))
        # # ********************************************************************************************保存SVM网格参数
        f = open(outPath + '/' + r'_svmParameter.txt', "w")
        f.write("SVM参数说明：\n")
        f.write('搜索网格参数1.核函数(kernel)\n')
        f.write('\t线性核函数(linear):主要用于线性可分，参数少，速度快\n')
        f.write('\t径向基核函数(rbf):主要用于线性不可分，参数多,耗时\n')
        f.write('\t多项式核函数(poly):依靠升维使得原本线性不可分的数据线性可分\n')
        f.write('\tSigmoid核函数:SVM实现的就是一种多层感知器神经网络\n')
        f.write('搜索网格参数2.惩罚参数(C)\n')
        f.write('\t[0.7, 0.8, 0.9, 1]\n\tC越大，松弛变量惩罚增大，即对误分类的惩罚增大，训练测试准确率高，但泛化能力弱；'
                'C值小，对误分类的惩罚减小，允许容错，将他们当成噪声点，泛化能力较强。\n')
        f.write('搜索最优网格参数:' + '\n' + str(grid_search.best_params_) + '\n')
        f.write('默认超参数:' + "\n" + str(best_model_svm) + '\n')
        f.close()

        y_scoreSVM1 = clf.predict(testData)
        # 预测标签
        y_scoreSVM = clf.predict_proba(testData)
        # 预测概率，计算样本点到分割超平面的函数距离,各标签预测概率[0.8,0.2]

        scoreSVM = clf.score(testData, testTarget)
        # 测试数据准去率
        # ORDER = [0, 1, 2, 3, 4]
        # classOneLabel
        matrixFindsize = 12
        # 混淆矩阵字体大小
        matrixDip = 120
        # 分辨率
        # Plot confusion matrix  绘图混淆矩阵
        sns.set_context("talk", rc={"font": "Helvetica", "font.size": matrixFindsize})
        label = [files[i] for i in classOneLabel]
        cm = confusion_matrix(testTarget, clf.predict(testData), labels=classOneLabel)
        plt.figure(figsize=(8, 6),
                   dpi=matrixDip,
                   # 分辨率
                   facecolor='snow',
                   # 背景色,'snow','blueviolet','lightgray'
                   )
        # 设置title
        plt.title('Test results of SVM classifier', fontsize=12)

        cm = 100 * cm / cm.sum(axis=1)[:, np.newaxis]
        ax = sns.heatmap(cm, annot_kws={'size': 8},
                         annot=True, cmap='Wistia', fmt='0.1f', xticklabels=label, yticklabels=label)
        # cmap的参数如下：Wistia（蓝绿黄）
        ax.xaxis.tick_top()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        # plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        # 设置刻度值即坐标轴刻度 xticks yticks
        plt.tick_params(labelsize=10)
        # 设置坐标轴的刻度参数
        outPath10 = outPath + '/' + r'_SVM.tif'
        plt.savefig(outPath10, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath10 = outPath + '/' + r'_SVM.pdf'
        plt.savefig(outPath10, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # ******************************************************************************************SVM的ROC曲线与AUC面积

        if 2 == len(classOneLabel):
            # 二分类ROC曲线
            fpr, tpr, threshold = roc_curve(testTarget, y_scoreSVM1)
            ###计算真正率和假正率
            roc_auc = auc(fpr, tpr)
            ###计算auc的值
            # Plot all ROC curves
            plt.figure()
            lw = 2
            plt.plot(fpr, tpr, color='darkorange',
                     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
            ###假正率为横坐标，真正率为纵坐标做曲线
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])

            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('SVM_ROC', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            outPath101 = outPath + '/' + r'_SVM_ROC.tif'
            plt.savefig(outPath101, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath101 = outPath + '/' + r'_SVM_ROC.pdf'
            plt.savefig(outPath101, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        else:
            # # 多分类ROC曲线
            # dataTarget1 = label_binarize(dataTarget, np.arange(len(classOneLabel)))
            # # 标签二值化，为了将ROC曲线和ROC区域扩展到多类或多标签分类，有必要对输出进行二值化：
            # # ⑴可以每个标签绘制一条ROC曲线。
            # # ⑵也可以通过将标签指示符矩阵的每个元素视为二元预测（微平均）来绘制ROC曲线。
            # # ⑶另一种用于多类别分类的评估方法是宏观平均，它对每个标签的分类给予相同的权重。
            #
            # print(np.array(dataTarget1).shape)
            #
            # trainData1, testData1, trainTarget1, testTarget1 = train_test_split(ySpec, dataTarget1,
            #                                                                     test_size=svmTestSize, random_state=1)

            # 计算每一类的ROC
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            # return 0
            for i in range(len(classOneLabel)):
                fpr[i], tpr[i], _ = roc_curve(testTarget1[:, i], y_scoreSVM[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area（方法二）
            fpr["micro"], tpr["micro"], _ = roc_curve(testTarget1.ravel(), y_scoreSVM.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # Compute macro-average ROC curve and ROC area（方法一）
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(classOneLabel))]))
            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in range(len(classOneLabel)):
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])
            # Finally average it and compute AU
            mean_tpr /= len(classOneLabel)
            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr
            roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

            # Plot all ROC curves
            plt.figure()
            plt.plot(fpr["micro"], tpr["micro"],
                     label='micro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["micro"]),
                     color='deeppink', linestyle=':')

            plt.plot(fpr["macro"], tpr["macro"],
                     label='macro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["macro"]),
                     color='navy', linestyle=':')

            # colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i in range(len(classOneLabel)):
                plt.plot(fpr[i], tpr[i],
                         label='ROC curve of class {0} (AUC = {1:0.2f})'
                               ''.format(files[i], roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('SVM_ROC:multi-class', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath101 = outPath + '/' + r'_SVM_ROC.tif'
            plt.savefig(outPath101, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath101 = outPath + '/' + r'_SVM_ROC.pdf'
            plt.savefig(outPath101, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        # ##**************************************************************************************************************LDA
        figure(3 * filesNumInter + 7)
        lda = LinearDiscriminantAnalysis()
        # print('中心点',lda.means_)

        # 建立需要搜索的参数的范围
        param_grid = {'n_components': [1,2, 5, 10]}
        grid_search_lda = GridSearchCV(lda, param_grid, cv=3)

        try:
            lda = grid_search_lda.fit(trainData, trainTarget)
        except:
            lda = grid_search_lda.fit(trainData, trainTarget)

        # lda = grid_search_lda.fit(trainData, trainTarget)
        # 训练模型
        # *******************************************************************************************保存、加载分类器模型
        # 保存模型
        outPathLDA = outPath + '/' + r'_ldaModel.pkl'
        joblib.dump(lda, outPathLDA)
        # 加载模型
        #
        best_model_lda = grid_search_lda.best_estimator_
        # print("***",type(grid_search.best_params_))
        # print(type(grid_search.best_params_))
        # print("默认超参数：",best_model)
        # print(type(best_model))
        # # ********************************************************************************************保存LDA网格参数
        f = open(outPath + '/' + r'_ldaParameter.txt', "w")
        f.write("LDA参数说明：\n")
        f.write('搜索网格参数1.解释器(solver)\n')
        f.write('\t奇异值分解(svd):对于有大规模特征的数据，推荐用这种算法\n')
        f.write('\t最小平方差(lsqr):可以结合skrinkage参数\n')
        f.write('\t特征分解算法(eigen):可以结合shrinkage参数\n')

        f.write('搜索网格参数2.指定了数组降维后的维度(n_components)\n')
        f.write('\t[2,3,5,8,12]\n')
        f.write('搜索最优网格参数:' + '\n' + str(grid_search_lda.best_params_) + '\n')
        f.write('默认超参数:' + "\n" + str(best_model_lda) + '\n')
        f.close()

        scoreLDA = lda.score(testData, testTarget)
        # 测试数据准去率
        # print('测试数据分类正确率', lda.score(X_test, Y_test))
        y_scoreLDA1 = lda.predict(testData)
        # 预测标签
        y_scoreLDA = lda.predict_proba(testData)
        # 预测标签概率

        matrixFindsize = 12
        # 混淆矩阵字体大小
        matrixDip = 120
        # 分辨率
        # Plot confusion matrix  绘图混淆矩阵
        sns.set_context("talk", rc={"font": "Helvetica", "font.size": matrixFindsize})
        label = [files[i] for i in classOneLabel]
        cm = confusion_matrix(testTarget, lda.predict(testData), labels=classOneLabel)
        plt.figure(figsize=(8, 6),
                   dpi=matrixDip,
                   # 分辨率
                   facecolor='blueviolet',
                   # 背景色,'snow','blueviolet','lightgray'
                   )
        # 设置title
        plt.title('Test results of LDA classifier', fontsize=12)

        cm = 100 * cm / cm.sum(axis=1)[:, np.newaxis]
        ax = sns.heatmap(cm, annot=True, cmap='YlOrRd', fmt='0.1f', xticklabels=label, yticklabels=label)
        ax.xaxis.tick_top()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        # plt.xticks()
        plt.yticks(rotation=0)
        # 设置刻度值即坐标轴刻度 xticks yticks
        plt.tick_params(labelsize=10)
        # 设置坐标轴的刻度参数
        outPath11 = outPath + '/' + r'_LDA.tif'
        plt.savefig(outPath11, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath11 = outPath + '/' + r'_LDA.pdf'
        plt.savefig(outPath11, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ******************************************************************************************LDA的ROC曲线与AUC面积

        if 2 == len(classOneLabel):
            # 二分类ROC曲线
            fpr, tpr, threshold = roc_curve(testTarget, y_scoreLDA1)
            ###计算真正率和假正率
            roc_auc = auc(fpr, tpr)
            ###计算auc的值
            # Plot all ROC curves
            plt.figure()
            lw = 2
            plt.plot(fpr, tpr, color='darkorange',
                     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
            ###假正率为横坐标，真正率为纵坐标做曲线
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])

            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('LDA_ROC', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath111 = outPath + '/' + r'_LDA_ROC.tif'
            plt.savefig(outPath111, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath111 = outPath + '/' + r'_LDA_ROC.pdf'
            plt.savefig(outPath111, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        else:
            #
            for i in range(len(classOneLabel)):
                fpr[i], tpr[i], _ = roc_curve(testTarget1[:, i], y_scoreLDA[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area（方法二）
            fpr["micro"], tpr["micro"], _ = roc_curve(testTarget1.ravel(), y_scoreLDA.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # Compute macro-average ROC curve and ROC area（方法一）
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(classOneLabel))]))
            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in range(len(classOneLabel)):
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])
            # Finally average it and compute AU
            mean_tpr /= len(classOneLabel)
            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr
            roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

            # Plot all ROC curves
            plt.figure()
            plt.plot(fpr["micro"], tpr["micro"],
                     label='micro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["micro"]),
                     color='deeppink', linestyle=':')

            plt.plot(fpr["macro"], tpr["macro"],
                     label='macro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["macro"]),
                     color='navy', linestyle=':')

            # colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i in range(len(classOneLabel)):
                plt.plot(fpr[i], tpr[i],
                         label='ROC curve of class {0} (AUC = {1:0.2f})'
                               ''.format(files[i], roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('LDA_ROC:multi-class', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath111 = outPath + '/' + r'_LDA_ROC.tif'
            plt.savefig(outPath111, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath111 = outPath + '/' + r'_LDA_ROC.pdf'
            plt.savefig(outPath111, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        # **************************************************************************************************************KNN
        figure(3 * filesNumInter + 8)

        knn = KNeighborsClassifier()

        # 定义一个knn分类器对象
        # int 型参数 knn算法中指定以最近的几个最近邻样本具有投票权，默认参数为5，n_neighbors=5
        # str参数,即每个拥有投票权的样本是按什么比重投票，'uniform'表示等比重投票，
        # 'distance'表示按距离反比投票，[callable]表示自己定义的一个函数
        # str或者距离度量对象   即怎样度量距离,默认是闵氏距离。

        # 建立需要搜索的参数的范围
        param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'leaf_size': [20, 30, 40]}
        grid_search_knn = GridSearchCV(knn, param_grid, cv=3)
        try:
            knn = grid_search_knn.fit(trainData, trainTarget)
        except:
            knn = grid_search_knn.fit(trainData, trainTarget)
        # knn = grid_search_knn.fit(trainData, trainTarget)
        # 训练模型
        # *******************************************************************************************保存、加载分类器模型
        # 保存模型
        outPathKNN = outPath + '/' + r'_knnModel.pkl'
        joblib.dump(knn, outPathKNN)
        # 加载模型
        #
        best_model_knn = grid_search_knn.best_estimator_
        # print("***",type(grid_search.best_params_))
        # print(type(grid_search.best_params_))
        # print("默认超参数：",best_model)
        # print(type(best_model))
        # # ********************************************************************************************保存knn网格参数
        f = open(outPath + '/' + r'_knnParameter.txt', "w")
        f.write("knn参数说明：\n")
        f.write('搜索网格参数1.相邻节点数(n_neighbors)\n')
        f.write('\t[3,5,7,10]\n')
        f.write('搜索网格参数2.权重函数(weights)\n')
        f.write('\tuniform:统一的权重. 在每一个邻居区域里的点的权重都是一样的。\n')
        f.write('\tdistance:权重点等于他们距离的倒数。使用此函数，更近的邻居对于所预测的点的影响更大。\n')
        f.write('搜索网格参数3.叶子数量(leaf_size)\n')
        f.write('\t[20,30,40]\n')
        f.write('搜索最优网格参数:' + '\n' + str(grid_search_knn.best_params_) + '\n')
        f.write('默认超参数:' + "\n" + str(best_model_knn) + '\n')
        f.close()

        # knn.fit(trainData, trainTarget)
        scoreKNN = knn.score(testData, testTarget)
        # 测试数据准确率
        y_scoreKNN1 = knn.predict(testData)
        # 预测标签
        y_scoreKNN = knn.predict_proba(testData)
        # 预测标签概率
        matrixFindsize = 12
        # 混淆矩阵字体大小
        matrixDip = 120
        # 分辨率
        # Plot confusion matrix  绘图混淆矩阵
        sns.set_context("talk", rc={"font": "Helvetica", "font.size": matrixFindsize})
        label = [files[i] for i in classOneLabel]
        cm = confusion_matrix(testTarget, knn.predict(testData), labels=classOneLabel)
        plt.figure(figsize=(8, 6),
                   dpi=matrixDip,
                   # 分辨率
                   facecolor='blueviolet',
                   # 背景色,'snow','blueviolet','lightgray'
                   )
        # 设置title
        plt.title('Test results of KNN classifier', fontsize=12)

        cm = 100 * cm / cm.sum(axis=1)[:, np.newaxis]
        ax = sns.heatmap(cm, annot=True, cmap='YlGnBu', fmt='0.1f', xticklabels=label, yticklabels=label)
        ax.xaxis.tick_top()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        # plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        # 设置刻度值即坐标轴刻度 xticks yticks
        plt.tick_params(labelsize=10)
        # 设置坐标轴的刻度参数
        outPath12 = outPath + '/' + r'_KNN.tif'
        plt.savefig(outPath12, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath12 = outPath + '/' + r'_KNN.pdf'
        plt.savefig(outPath12, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # ******************************************************************************************KNN的ROC曲线与AUC面积
        if 2 == len(classOneLabel):
            # 二分类ROC曲线
            fpr, tpr, threshold = roc_curve(testTarget, y_scoreKNN1)
            ###计算真正率和假正率
            roc_auc = auc(fpr, tpr)
            ###计算auc的值
            # Plot all ROC curves
            plt.figure()
            lw = 2
            plt.plot(fpr, tpr, color='darkorange',
                     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
            ###假正率为横坐标，真正率为纵坐标做曲线
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])

            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('KNN_ROC', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath121 = outPath + '/' + r'_KNN_ROC.tif'
            plt.savefig(outPath121, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath121 = outPath + '/' + r'_KNN_ROC.pdf'
            plt.savefig(outPath121, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        else:
            # 多分类
            for i in range(len(classOneLabel)):
                fpr[i], tpr[i], _ = roc_curve(testTarget1[:, i], y_scoreKNN[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area（方法二）
            fpr["micro"], tpr["micro"], _ = roc_curve(testTarget1.ravel(), y_scoreKNN.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # Compute macro-average ROC curve and ROC area（方法一）
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(classOneLabel))]))
            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in range(len(classOneLabel)):
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])
            # Finally average it and compute AU
            mean_tpr /= len(classOneLabel)
            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr
            roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

            # Plot all ROC curves
            plt.figure()
            plt.plot(fpr["micro"], tpr["micro"],
                     label='micro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["micro"]),
                     color='deeppink', linestyle=':')

            plt.plot(fpr["macro"], tpr["macro"],
                     label='macro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["macro"]),
                     color='navy', linestyle=':')

            # colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i in range(len(classOneLabel)):
                plt.plot(fpr[i], tpr[i],
                         label='ROC curve of class {0} (AUC = {1:0.2f})'
                               ''.format(files[i], roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('KNN_ROC:multi-class', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath121 = outPath + '/' + r'_KNN_ROC.tif'
            plt.savefig(outPath121, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath121 = outPath + '/' + r'_KNN_ROC.pdf'
            plt.savefig(outPath121, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()

        # **************************************************************************************************************XGBoost
        figure(3 * filesNumInter + 8)

        xgb = XGBClassifier()
        # scale_pos_weight,正样本的权重，在二分类任务中，当正负样本比例失衡时，设置正样本的权重，模型效果更好。
        # 建立需要搜索的参数的范围

        param_grid = {'max_depth': [5, 6, 7]}
        grid_search_xgb = GridSearchCV(xgb, param_grid, cv=3)
        try:
            xgb = grid_search_xgb.fit(trainData, trainTarget)
        except:
            xgb = grid_search_xgb.fit(trainData, trainTarget)
        # xgb = grid_search_xgb.fit(trainData, trainTarget)
        # 训练模型
        # *******************************************************************************************保存、加载分类器模型
        # 保存模型
        outPathXGB = outPath + '/' + r'_xgbModel.pkl'
        joblib.dump(xgb, outPathXGB)
        # 加载模型
        best_model_xgb = grid_search_xgb.best_estimator_
        # 网格函数选出的最优模型
        # print("***",type(grid_search.best_params_))
        # print(type(grid_search.best_params_))
        # print("默认超参数：",best_model)
        # print(type(best_model))
        # **************************************************************************************************************XGB特征重要性(gain增益)排序条形图
        '''
        # 重要性指标feature_importances_，这个参数好像只有在决策树和以决策树为基础的算法有。
        '''
        top_K = 20
        # 返回前n个特征
        # xgbFeatureImportance = XGBClassifier()
        xgbFeaImp = best_model_xgb.fit(trainData, trainTarget).feature_importances_
        top_k_idx = np.array(xgbFeaImp).argsort()[::-1][0:top_K]
        # argsort:返回的是数组值从小到大的索引值
        f = open(outPath + '/' + r'_xgbGain.txt', "w")
        f.write('用XGBoost模型对光谱特征重要性进行排序：输出前10最重要拉曼位移(特征)及重要性占比' + '\n')
        for interI in range(top_K):
            f.write("拉曼位移:" + str(x[top_k_idx[interI]]) + ", 占比：" + str(round(xgbFeaImp[top_k_idx[interI]], 3)) + '\n')

        f.write('注：属性(拉曼位移对应的波数点)对分裂点改进性能度量越大（越靠近根节点），权值越大；\n'
                '被越多提升树所选择，属性越重要。这里性能度量选择分裂节点的gain增益。' + '\n')
        f.close()

        wavenumberX = np.arange(len(top_k_idx))
        # 添加波数坐标
        plt.bar(x=wavenumberX, height=xgbFeaImp[top_k_idx], width=0.5, tick_label=x[top_k_idx], color='#00BFFF')
        plt.xticks(fontsize=9, rotation=45)
        plt.yticks(fontsize=10)
        plt.xlabel("Characteristic peak position", fontsize=12)
        plt.ylabel("Gain", fontsize=12)
        plt.tight_layout()
        # 解决显示不全的问题
        # plt.show()

        plt.gca().yaxis.set_major_formatter(FuncFormatter(to_percent))
        # from matplotlib import ticker
        # plt.gca().yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=1))

        # 纵坐标以百分百%显示
        outPath123 = outPath + '/' + r'_XGB_Gain.tif'
        plt.savefig(outPath123, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath123 = outPath + '/' + r'_XGB_Gain.pdf'
        plt.savefig(outPath123, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()
        # return 0
        # # ********************************************************************************************保存xgb网格参数
        f = open(outPath + '/' + r'_xgbParameter.txt', "w")
        f.write("xgb参数说明：\n")
        f.write('搜索网格参数1.基分类器模型(booster)\n')
        f.write('\tgbtree:基分类器为树模型;gbliner:基分类器为线性模型\n')
        f.write('搜索网格参数2.树的深度(max_depth)\n')
        f.write('\t[5,6,7]值过大容易过拟合，值过小容易欠拟合\n')
        f.write('\tdistance:权重点等于他们距离的倒数。使用此函数，更近的邻居对于所预测的点的影响更大。\n')
        f.write('搜索网格参数3.基分类器的个数(n_estimatores)\n')
        f.write('\t[80, 100, 120]\n')
        f.write('搜索最优网格参数:' + '\n' + str(grid_search_xgb.best_params_) + '\n')
        f.write('默认超参数:' + "\n" + str(best_model_xgb) + '\n')
        f.close()

        # xgb.fit(trainData, trainTarget)
        scoreXGB = xgb.score(testData, testTarget)
        # 测试数据准确率
        y_scoreXGB1 = xgb.predict(testData)
        # 预测标签
        y_scoreXGB = xgb.predict_proba(testData)
        # 预测标签概率
        matrixFindsize = 12
        # 混淆矩阵字体大小
        matrixDip = 120
        # 分辨率
        # Plot confusion matrix  绘图混淆矩阵
        sns.set_context("talk", rc={"font": "Helvetica", "font.size": matrixFindsize})
        label = [files[i] for i in classOneLabel]
        cm = confusion_matrix(testTarget, xgb.predict(testData), labels=classOneLabel)
        plt.figure(figsize=(8, 6),
                   dpi=matrixDip,
                   # 分辨率
                   facecolor='blueviolet',
                   # 背景色,'snow','blueviolet','lightgray'
                   )
        # 设置title
        plt.title('Test results of XGB classifier', fontsize=12)

        cm = 100 * cm / cm.sum(axis=1)[:, np.newaxis]
        ax = sns.heatmap(cm, annot=True, cmap='YlGnBu', fmt='0.1f', xticklabels=label, yticklabels=label)
        ax.xaxis.tick_top()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        # plt.xticks(rotation=90)
        plt.yticks(rotation=0)
        # 设置刻度值即坐标轴刻度 xticks yticks
        plt.tick_params(labelsize=10)
        # 设置坐标轴的刻度参数
        outPath122 = outPath + '/' + r'_XGB.tif'
        plt.savefig(outPath122, format='tif', bbox_inches='tight', transparent=True, dpi=300)
        outPath122 = outPath + '/' + r'_XGB.pdf'
        plt.savefig(outPath122, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
        plt.close()

        # *************************************************************************************XGB的ROC曲线与AUC面积
        if 2 == len(classOneLabel):
            # 二分类ROC曲线
            fpr, tpr, threshold = roc_curve(testTarget, y_scoreXGB1)
            ###计算真正率和假正率
            roc_auc = auc(fpr, tpr)
            ###计算auc的值
            # Plot all ROC curves
            plt.figure()
            lw = 2
            plt.plot(fpr, tpr, color='darkorange',
                     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
            ###假正率为横坐标，真正率为纵坐标做曲线
            plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])

            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('XGB_ROC', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath121 = outPath + '/' + r'_XGB_ROC.tif'
            plt.savefig(outPath121, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath121 = outPath + '/' + r'_XGB_ROC.pdf'
            plt.savefig(outPath121, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()
        else:
            # 多分类
            for i in range(len(classOneLabel)):
                fpr[i], tpr[i], _ = roc_curve(testTarget1[:, i], y_scoreXGB[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area（方法二）
            fpr["micro"], tpr["micro"], _ = roc_curve(testTarget1.ravel(), y_scoreXGB.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # Compute macro-average ROC curve and ROC area（方法一）
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(classOneLabel))]))
            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in range(len(classOneLabel)):
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])
            # Finally average it and compute AU
            mean_tpr /= len(classOneLabel)
            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr
            roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

            # Plot all ROC curves
            plt.figure()
            plt.plot(fpr["micro"], tpr["micro"],
                     label='micro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["micro"]),
                     color='deeppink', linestyle=':')

            plt.plot(fpr["macro"], tpr["macro"],
                     label='macro-average ROC curve (AUC = {0:0.2f})'
                           ''.format(roc_auc["macro"]),
                     color='navy', linestyle=':')

            # colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i in range(len(classOneLabel)):
                plt.plot(fpr[i], tpr[i],
                         label='ROC curve of class {0} (AUC = {1:0.2f})'
                               ''.format(files[i], roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('XGB_ROC:multi-class', fontsize=12)
            plt.legend(loc="lower right", fontsize=8)
            # plt.show()
            outPath121 = outPath + '/' + r'_XGB_ROC.tif'
            plt.savefig(outPath121, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath121 = outPath + '/' + r'_XGB_ROC.pdf'
            plt.savefig(outPath121, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()

        # **************************************************************************************************************回归分析画图(小于5类)

        if filesNumInter <= 4:

            # ySpecL = pd.DataFrame(ySpec)
            # dataTargetL = pd.Series(dataTarget)
            # # 数据类型转换
            # trainDataL, testDataL, trainTargetL, testTargetL = train_test_split(ySpecL, dataTargetL,
            #                                                                     test_size=svmTestSize, random_state=1)
            # # 保证随机种子一致random_state=1
            testTargetL = pd.Series(testTarget)
            labelTest = list(testTargetL.index)
            # labelTest = list(testTarget.index)

            # labelTest.sort()
            # # 测试数据索引标签排序
            xLabelTest = []
            yLabelTest = []
            for interLabelTest in labelTest:
                xLabelTest.append(result2[interLabelTest, pca_col1])
                yLabelTest.append(result2[interLabelTest, pca_col2])

            outPuthExc = outPath + '/' + r'Classification.csv'
            allDataTwoClass = np.column_stack(
                (np.array(xLabelTest), np.array(yLabelTest), np.array(testTarget), np.array(knn.predict(testData)),
                 np.array(knn.predict(testData)), np.array(clf.predict(testData))))
            allDataTwoClass1 = pd.DataFrame(allDataTwoClass,
                                            columns=['x_tSNE1', 'y_tSNE2', 'trueLabel', 'svmClassify', 'ldaClassify',
                                                     'knnClassify'])
            allDataTwoClass1.to_csv(outPuthExc)

            df = pd.read_csv(outPuthExc)
            # **************************************************************************************************SVM二分类
            sns.set_style("white")
            gridobj = sns.lmplot(x="x_tSNE1", y="y_tSNE2", hue="svmClassify", data=df,
                                 height=5, aspect=1.6, palette='tab10',
                                 scatter_kws=dict(s=20, linewidths=.7, edgecolors='black'))
            # gridobj = sns.lmplot(x="x_tSNE1", y="y_tSNE2", hue="svmClassify", data=df,
            #                      height=5, aspect=1.6, robust=True, palette='tab10',
            #                      scatter_kws=dict(s=20, linewidths=.7, edgecolors='black'))
            # Decorations
            # gridobj.set(xlim=(0.5, 7.5), ylim=(0, 50))
            gridobj.set()
            plt.title("Scatterplot with line of best fit - SVM", fontsize=10)
            # plt.show()
            outPath13 = outPath + '/' + r'_TwoCategoriesSVM.tif'
            plt.savefig(outPath13, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath13 = outPath + '/' + r'_TwoCategoriesSVM.pdf'
            plt.savefig(outPath13, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()

            # **************************************************************************************************LDA二分类
            sns.set_style("white")
            gridobj = sns.lmplot(x="x_tSNE1", y="y_tSNE2", hue="ldaClassify", data=df,
                                 height=5, aspect=1.6, palette='tab10',
                                 scatter_kws=dict(s=20, linewidths=.7, edgecolors='black'))

            # Decorations
            # gridobj.set(xlim=(0.5, 7.5), ylim=(0, 50))
            gridobj.set()
            plt.title("Scatterplot with line of best fit - LDA", fontsize=10)
            # plt.show()
            outPath14 = outPath + '/' + r'_TwoCategoriesLDA.tif'
            plt.savefig(outPath14, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath14 = outPath + '/' + r'_TwoCategoriesLDA.pdf'
            plt.savefig(outPath14, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close()

            # **************************************************************************************************KNN二分类
            sns.set_style("white")
            gridobj = sns.lmplot(x="x_tSNE1", y="y_tSNE2", hue="knnClassify", data=df,
                                 height=5, aspect=1.6, palette='tab10',
                                 scatter_kws=dict(s=20, linewidths=.7, edgecolors='black'))
            # Decorations
            # gridobj.set(xlim=(0.5, 7.5), ylim=(0, 50))
            gridobj.set()
            plt.title("Scatterplot with line of best fit - KNN", fontsize=10)
            # plt.show()
            outPath15 = outPath + '/' + r'_TwoCategoriesKNN.tif'
            plt.savefig(outPath15, format='tif', bbox_inches='tight', transparent=True, dpi=300)
            outPath15 = outPath + '/' + r'_TwoCategoriesKNN.pdf'
            plt.savefig(outPath15, format='pdf', bbox_inches='tight', transparent=True, dpi=300)
            plt.close("all")
        # ********************************************************************************************************************写入word
        # 输出信息
        dataNum = allData.shape[1] - 1
        # 各标签数据量
        # outPutInfo = [dataNum, files, pcaCountAll, snrDelfile]

        document = Document()
        # 打开文档
        document.styles['Normal'].font.name = u'宋体'
        document.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
        # 设置文档格式
        # t0：标题
        t0 = document.add_heading(u'实验报告', 1)
        t0.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # 直接添加标题，“1”表示1级标题,标题居中
        # t1,t2,t3次级标题
        t1 = document.add_heading(u'1.数据信息', 2)
        # 增加表格1
        rows_num = 3
        cols_num = filesNumInter + 1
        table1 = document.add_table(rows=rows_num, cols=cols_num, style='Table Grid')
        table1.style.font.name = u'宋体'
        listTable1 = ['标签名', '数据量', '剔除光谱编号']
        # 列标签
        for i in range(rows_num):
            table1.cell(i, 0).text = listTable1[i]
        # 行信息
        for j in range(1, cols_num):
            table1.cell(0, j).text = files[j - 1]
            table1.cell(1, j).text = str(pcaCountAll[j - 1])
            table1.cell(2, j).text = str(snrDelfile[j - 1])
        t2 = document.add_heading(u'2.参数信息', 2)
        # 增加表格2
        listTable2 = ["参数名", "参数意义", "参数值"]
        listTable2Name = ["dataPath", "outPath", "startSpec", "endSpec", "snrNoiseindex", "delSNRmin", "delSNRmax",
                          "filterSize",
                          "dynamicFactor", "winSG", "nSG", "lambdaAirPls", "itermaxAirPls", "pltInter",
                          "processLinewidth",
                          "distanceMethod", "hcaLabelsize", "pcaNcomponents", "pca_col1", "pca_col2", "pcaSize",
                          "pcaAlpha",
                          "nDim", "nPerplex", "learnRate", "nInter", "nGrad", "svmTestSize", "svmC", "xKernel",
                          "clusters_N"]
        listTable2Meaning = ["输入路径", "输出路径", "截谱长度(首)", "截谱长度(尾)", "静默区：尾端长度", "信噪比最小阈值", "信噪比最大阈值",
                             "宇宙射线：窗口宽度", "宇宙射线：动态因子", "滤波：窗口宽度", "滤波：拟合阶次",
                             "基线校正：lambda", "基线校正：迭代次数", "显示光谱量(未使用)", "光谱线宽(未使用)",
                             "HCA距离度量方法:\r0:沃德方差最小化\r1:质心欧式距离\r2:加权分组平均\r3:UPGMA算法",
                             "HCA(tSNE)标号大小", "PCA主成分个数", "PCA(tSNE)选择维度", "PCA(tSNE)选择维度",
                             "PCA(tSNE)标签大小", "PCA(tSNE)透明度", "tSNE空间维度", "Perplex", "tSNE学习率",
                             "tSNE最大迭代次数", "tSNE收敛误差", "测试数据占比", "SVM错误项惩罚系数",
                             "SVM核函数:\r0:多项式核\r1:线性核\r2:径向基核\r3:sigmod核\r4:核矩阵", "聚类堆数"]
        listTable2Value = [dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize,
                           dynamicFactor, winSG,
                           nSG, lambdaAirPls, itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize,
                           pcaNcomponents, pca_col1, pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter,
                           nGrad,
                           svmTestSize, svmC, xKernel, clusters_N]
        rows_num = len(listTable2Meaning) + 1
        cols_num = len(listTable2)
        table2 = document.add_table(rows=rows_num, cols=cols_num, style='Table Grid')
        table2.style.font.name = u'宋体'
        # 列标题
        for j in range(0, cols_num):
            table2.cell(0, j).text = listTable2[j]
        # 行标题
        for i in range(1, rows_num):
            table2.cell(i, 0).text = listTable2Name[i - 1]
            table2.cell(i, 1).text = listTable2Meaning[i - 1]
            table2.cell(i, 2).text = str(listTable2Value[i - 1])
        # 均值方差图
        t3 = document.add_heading(u'3.均值方差图', 2)
        for i in range(filesNumInter):
            document.add_heading(files[i] + '_光谱均值_信噪比标准差', 3)
            img_name = outPath + '/' + files[i] + r'_meanStd.tif'
            fig2 = document.add_picture(img_name, width=Inches(5.0))
            # 向文档里添加图片
            # fig2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            # #居中显示
        # 均值方差罗列图
        t4 = document.add_heading(u'4.均值方差罗列图', 2)
        img_name = outPath + '/' + r'_meanStdcolumn.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # 信噪比箱型图
        t5 = document.add_heading(u'5.信噪比箱型图', 2)
        img_name = outPath + '/' + r'_snrBox.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # 信噪比直方图
        t6 = document.add_heading(u'6.信噪比直方图', 2)
        img_name = outPath + '/' + r'_snrHistogram.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # PCA
        t7 = document.add_heading(u'7.PCA主成分分布图', 2)
        img_name = outPath + '/' + r'_PCA_3D.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # tSNE
        t8 = document.add_heading(u'8.tSNE空间分布图', 2)
        img_name = outPath + '/' + r'_tSNE.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # HCA
        t9 = document.add_heading(u'9.层次聚类树状图', 2)
        img_name = outPath + '/' + r'_HCA.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # SV
        t10 = document.add_heading(u'10.混淆矩阵：SVM测试结果', 2)
        table10 = document.add_table(rows=1, cols=2, style='Table Grid')
        table10.style.font.name = u'宋体'
        table10.cell(0, 0).text = "SVM测试准确率"
        table10.cell(0, 1).text = str("%.2f%%" % (scoreSVM * 100))
        img_name = outPath + '/' + r'_SVM.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # SVM_ROC_AUC
        t11 = document.add_heading(u'10.ROC评估：SVM的ROC曲线及AUC值', 2)
        table11 = document.add_table(rows=3, cols=2, style='Table Grid')
        table11.style.font.name = u'宋体'
        table11.cell(0, 0).text = "macro-average ROC curve"
        table11.cell(0, 1).text = "类别维度，n个类别ROC曲线均值"
        table11.cell(1, 0).text = "micro-average ROC curve"
        table11.cell(1, 1).text = "样本维度，计算所有测试样本ROC曲线均值"
        table11.cell(2, 0).text = "ROC curve of class * "
        table11.cell(2, 1).text = "类别 * 与SVM分类器预测概率绘制的ROC曲线"
        img_name = outPath + '/' + r'_SVM_ROC.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # SVM两类
        if filesNumInter <= 4:
            t101 = document.add_heading(u'10(2).最佳回归线散点图：SVM测试结果', 2)
            img_name = outPath + '/' + r'_TwoCategoriesSVM.tif'
            document.add_picture(img_name, width=Inches(5.0))
        # KNN
        t12 = document.add_heading(u'11.混淆矩阵：KNN测试结果', 2)
        table12 = document.add_table(rows=1, cols=2, style='Table Grid')
        table12.style.font.name = u'宋体'
        table12.cell(0, 0).text = "KNN测试准确率"
        table12.cell(0, 1).text = str("%.2f%%" % (scoreKNN * 100))
        img_name = outPath + '/' + r'_KNN.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # KNN_ROC_AUC
        t13 = document.add_heading(u'10.ROC评估：KNN的ROC曲线及AUC值', 2)
        table13 = document.add_table(rows=3, cols=2, style='Table Grid')
        table13.style.font.name = u'宋体'
        table13.cell(0, 0).text = "macro-average ROC curve"
        table13.cell(0, 1).text = "类别维度，n个类别ROC曲线均值"
        table13.cell(1, 0).text = "micro-average ROC curve"
        table13.cell(1, 1).text = "样本维度，计算所有测试样本ROC曲线均值"
        table13.cell(2, 0).text = "ROC curve of class * "
        table13.cell(2, 1).text = "类别 * 与KNN分类器预测概率绘制的ROC曲线"
        img_name = outPath + '/' + r'_KNN_ROC.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # KNN两类-四类
        if filesNumInter <= 4:
            t101 = document.add_heading(u'11(2).最佳回归线散点图：KNN测试结果', 2)
            img_name = outPath + '/' + r'_TwoCategoriesKNN.tif'
            document.add_picture(img_name, width=Inches(5.0))
        # LDA
        t14 = document.add_heading(u'12.混淆矩阵：LDA测试结果', 2)
        table14 = document.add_table(rows=1, cols=2, style='Table Grid')
        table14.style.font.name = u'宋体'
        table14.cell(0, 0).text = "测试准确率"
        table14.cell(0, 1).text = str("%.2f%%" % (scoreLDA * 100))
        img_name = outPath + '/' + r'_LDA.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # LDA_ROC_AUC
        t15 = document.add_heading(u'10.ROC评估：LDA的ROC曲线及AUC值', 2)
        table15 = document.add_table(rows=3, cols=2, style='Table Grid')
        table15.style.font.name = u'宋体'
        table15.cell(0, 0).text = "macro-average ROC curve"
        table15.cell(0, 1).text = "类别维度，n个类别ROC曲线均值"
        table15.cell(1, 0).text = "micro-average ROC curve"
        table15.cell(1, 1).text = "样本维度，计算所有测试样本ROC曲线均值"
        table15.cell(2, 0).text = "ROC curve of class * "
        table15.cell(2, 1).text = "类别 * 与LDA分类器预测概率绘制的ROC曲线"
        img_name = outPath + '/' + r'_LDA_ROC.tif'
        document.add_picture(img_name, width=Inches(5.0))
        # LDA两类-四类
        if filesNumInter <= 4:
            t101 = document.add_heading(u'12(2).最佳回归线散点图：LDA测试结果', 2)
            img_name = outPath + '/' + r'_TwoCategoriesLDA.tif'
            document.add_picture(img_name, width=Inches(5.0))

        document.save(outPath + '/' + r"HookeReport_Multifile.docx")
        # 保存文档
        # ********************************************************************************************************************删除文件夹下所有tif图片

        # del_files(outPath)

        return 1



In [5]:
# # # #
# 调用测试
# 调用测试
#dataPath2 = r'./结果/4/输入数据/12-基于SVM模型的本次结果的预测-阴性对照/mapping10'
dataPath = r'./data/消化道肿瘤-插值后'
outPath =r'./result/消化道肿瘤-插值后'
# 截谱
startSpec = 80
# endSpec = 0
endSpec = 0
# 前处理参数
snrNoiseindex = 20
# 从尾部选取长度，作为静默区

delSNRmin = 0
# 删除光谱信噪比阈值
delSNRmax = 10000
# 删除光谱信噪比阈值

# 批处理参数
# 2.1 宇宙射线阈值
filterSize = 5
# 整型
dynamicFactor = 4.5
# 浮点型
# 2.2 S-G滤波
winSG = 5
nSG = 3
lambdaAirPls = 100
itermaxAirPls = 15
# 更保型
#lambdaAirPls = 100
# itermaxAirPls = 15
pltInter = 5
# 画光谱数量
processLinewidth = 0.5
# 线宽

# hca参数
distanceMethod = 0
# 层次聚类距离度量选用的方法,可选：'centroid','weighted','average'
# hcaColorThreshold = 10
# # color_threshold= 8 设置颜色高度阈值，用于控制决策树和PCA的颜色分类
hcaLabelsize = 8
# 标号大小

# pca参数,和tsne共用参数
pcaNcomponents = 2
# 主成分个数，大于等于2
pca_col1 = 0
# 第一维，小于主成分个数
pca_col2 = 1
# 第二维，小于主成分个数
pcaSize = 30
# pca标签大小
pcaAlpha = 1
###############################################################20220301改的，之后改回来#pcaAlpha = 0.8
# pca透明度

# tSNE参数
nDim = 2
# int, 空间的维度
nPerplex = 8
# 数据集越大，需要参数值越大，建议值位5 - 50
learnRate = 200
# float,学习率，建议取值为10.0-1000.0
nInter = 1000
# int, 最大迭代次数
nGrad = 1e-06
# float,如果梯度低于该值，则停止算法

# SVM参数
svmTestSize = 0.2
# 测试数据占全部数据比例
svmC = 0.8
# 错误项的惩罚系数,范围[0.5-1],C越大分类效果越好，但有可能会过拟合（defaul C = 1）
xKernel = 0
# str参数,算法中采用的核函数类型,可选'linear'、'poly'、'rbf'、'sigmoid'、'precomputed'，默认为rbf
clusters_N = 4
# 聚类堆数
oneclickReport(dataPath, outPath, startSpec, endSpec, snrNoiseindex, delSNRmin, delSNRmax, filterSize, dynamicFactor,
               winSG, nSG, lambdaAirPls,
               itermaxAirPls, pltInter, processLinewidth, distanceMethod, hcaLabelsize, pcaNcomponents,
               pca_col1, pca_col2, pcaSize, pcaAlpha, nDim, nPerplex, learnRate, nInter, nGrad, svmTestSize,
               svmC, xKernel, clusters_N)


1

In [12]:
import torch
print(torch.__version__)  #注意是双下划线

1.8.0
